# Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import networkx as nx
from collections import defaultdict, Counter, deque
import matplotlib.pyplot as plt
from numba import jit, prange
import time
from typing import Dict, Tuple, Optional
import warnings
import scipy.sparse as sp
from datetime import datetime
import pickle
warnings.filterwarnings('ignore')

# Inputs

In [2]:
EN_PATH = '../Data/Input/conceptnet_en_full.csv'

# Purpose
***

Allow an agent to interact with the full relation universe and create a full knowledge graph.

### Phase 1. 
***
Load the english triples

In [3]:
english_triples = pd.read_csv(EN_PATH)
english_triples

,uri,relation,start,end,weight,dataset,sources,surfaceText,license,context
0,"/a/[/r/Antonym/,/c/en/0/n/,/c/en/1/]",/r/Antonym,/c/en/0/n,/c/en/1,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
1,"/a/[/r/Antonym/,/c/en/12_hour_clock/n/,/c/en/2...",/r/Antonym,/c/en/12_hour_clock/n,/c/en/24_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
2,"/a/[/r/Antonym/,/c/en/24_hour_clock/n/,/c/en/1...",/r/Antonym,/c/en/24_hour_clock/n,/c/en/12_hour_clock,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
3,"/a/[/r/Antonym/,/c/en/5/n/,/c/en/3/]",/r/Antonym,/c/en/5/n,/c/en/3,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
4,"/a/[/r/Antonym/,/c/en/a.c/n/,/c/en/d.c/]",/r/Antonym,/c/en/a.c/n,/c/en/d.c,"{""dataset"": ""/d/wiktionary/fr"", ""license"": ""cc...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3474741,"/a/[/r/dbpedia/product/,/c/en/xerox/,/c/en/pro...",/r/dbpedia/product,/c/en/xerox,/c/en/projector,"{""dataset"": ""/d/dbpedia/en"", ""license"": ""cc:by...",NaN,NaN,NaN,NaN,NaN
3474742,"/a/[/r/dbpedia/product/,/c/en/zanella/,/c/en/m...",/r/dbpedia/product,/c/en/zanella,/c/en/moped,"{""dataset"": ""/d/dbpedia/en"", ""license"": ""cc:by...",NaN,NaN,NaN,NaN,NaN
3474743,"/a/[/r/dbpedia/product/,/c/en/zanella/,/c/en/m...",/r/dbpedia/product,/c/en/zanella,/c/en/motorcycle,"{""dataset"": ""/d/dbpedia/en"", ""license"": ""cc:by...",NaN,NaN,NaN,NaN,NaN
3474744,"/a/[/r/dbpedia/product/,/c/en/zara/n/wp/retail...",/r/dbpedia/product,/c/en/zara/n/wp/retailer,/c/en/clothing,"{""dataset"": ""/d/dbpedia/en"", ""license"": ""cc:by...",NaN,NaN,NaN,NaN,NaN


#### Phase 1.A 
***
Preprocess and prepare the concept data

In [4]:
def clean_column(df, col):
    col_string = df[col].astype(str)
    col_string = col_string.str.split('/')
    col_string = col_string.str[-1]
    df[f'{col}_cleaned'] = col_string
    return df

def extract_weight(df):
    df['weight_cleaned'] = df['weight'].apply(lambda x: json.loads(x)['weight'])
    return df

def preprocess_data(df):
    df_copy = df.copy()
    for col in ['relation', 'start', 'end']:
        df_copy = clean_column(df_copy, col)
    df_copy = extract_weight(df_copy)
    return df_copy

cleaned_cols = ['relation', 'start', 'end', 'weight']
cleaned_cols = [f'{col}_cleaned' for col in cleaned_cols]

reprocess_triples = False

if reprocess_triples:
    cleaned_english_triples = preprocess_data(english_triples)[cleaned_cols].drop_duplicates()
    cleaned_english_triples[cleaned_cols].head()
    cleaned_english_triples = cleaned_english_triples.rename(columns={
        'relation_cleaned': 'relation_type',
        'start_cleaned': 'start_concept',
        'end_cleaned': 'end_concept',
        'weight_cleaned': 'edge_weight'
    })
    # Save as a parquet file
    cleaned_english_triples.to_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_cleaned.parquet.gzip'),
        index=False,
        compression='gzip'
    )
else:
    # Load the cleaned data
    cleaned_english_triples = pd.read_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_cleaned.parquet.gzip')
    )
cleaned_english_triples.head()

,relation_type,start_concept,end_concept,edge_weight
0,Antonym,n,1,1.0
1,Antonym,n,24_hour_clock,1.0
2,Antonym,n,12_hour_clock,1.0
3,Antonym,n,3,1.0
4,Antonym,n,d.c,1.0


In [5]:
print(f"Number of unique start concepts: {len(cleaned_english_triples['start_concept'].unique())}")
print(f"Number of unique end concepts: {len(cleaned_english_triples['end_concept'].unique())}")
print(f"Number of unique relation types: {len(cleaned_english_triples['relation_type'].unique())}")
print(f'Size of the dataset: {len(cleaned_english_triples)}')

Number of unique start concepts: 374417
Number of unique end concepts: 602177
Number of unique relation types: 47
Size of the dataset: 1655522


# Phase 2. Seed the agent with random english relations
***

In [6]:
def create_stratified_weighted_sample(df, sample_size=5000, min_weight_threshold=0.5, verbose=True):
    """
    Create a stratified sample maintaining relation distribution while prioritizing higher weights
    
    Parameters:
    - df: DataFrame with columns ['start_concept', 'end_concept', 'relation_type', 'edge_weight']
    - sample_size: Target number of triples in sample
    - min_weight_threshold: Minimum weight to consider (filters low-quality relations)
    - verbose: Print detailed progress
    
    Returns:
    - DataFrame: Stratified sample
    """
    
    # df_copy = df.copy()
    # df_copy = df_copy.rename(columns={
    #     'relation_cleaned': 'relation_type',
    #     'start_cleaned': 'start_concept',
    #     'end_cleaned': 'end_concept',
    #     'weight_cleaned': 'edge_weight'
    # })
    
    print(f"🎯 Creating stratified weighted sample of {sample_size:,} triples...")
    
    # Step 1: Filter by minimum weight threshold
    if verbose:
        print(f"   Filtering triples with weight >= {min_weight_threshold}")
    
    initial_count = len(df)
    filtered_df = df[df['edge_weight'] >= min_weight_threshold].copy()
    filtered_count = len(filtered_df)
    
    if verbose:
        print(f"   Kept {filtered_count:,} of {initial_count:,} triples ({filtered_count/initial_count*100:.1f}%)")
    
    # Step 2: Calculate current relation distribution
    relation_counts = filtered_df['relation_type'].value_counts()
    relation_proportions = relation_counts / len(filtered_df)
    
    if verbose:
        print(f"\n📊 Original relation distribution (top 10):")
        for relation, prop in relation_proportions.head(10).items():
            count = relation_counts[relation]
            print(f"   {relation}: {count:,} ({prop*100:.1f}%)")
    
    # Step 3: Sort by weight within each relation (highest first)
    if verbose:
        print(f"\n⚖️  Sorting by weight within each relation...")
    
    filtered_df = filtered_df.sort_values(['relation_type', 'edge_weight'], 
                                        ascending=[True, False])
    
    # Step 4: Calculate target samples per relation
    target_samples_per_relation = {}
    for relation in relation_proportions.index:
        target_count = int(sample_size * relation_proportions[relation])
        # Ensure at least 1 sample for each relation if possible
        target_count = max(1, target_count)
        target_samples_per_relation[relation] = target_count
    
    if verbose:
        print(f"\n🎯 Target samples per relation:")
        total_targeted = sum(target_samples_per_relation.values())
        for relation, target in sorted(target_samples_per_relation.items(), 
                                     key=lambda x: x[1], reverse=True)[:10]:
            print(f"   {relation}: {target:,}")
        print(f"   Total targeted: {total_targeted:,}")
    
    # Step 5: Sample from each relation group
    sampled_dfs = []
    actual_samples = {}
    
    if verbose:
        print(f"\n🔄 Sampling from each relation group...")
    
    for relation, target_count in tqdm(target_samples_per_relation.items(), 
                                     desc="Sampling relations"):
        relation_data = filtered_df[filtered_df['relation_type'] == relation]
        
        # Take top weighted samples up to target count
        actual_count = min(target_count, len(relation_data))
        sampled_data = relation_data.head(actual_count)
        
        sampled_dfs.append(sampled_data)
        actual_samples[relation] = actual_count
    
    # Step 6: Combine all samples
    stratified_sample = pd.concat(sampled_dfs, ignore_index=True)
    
    # Step 7: If we're short, fill with highest-weight remaining samples
    current_size = len(stratified_sample)
    if current_size < sample_size:
        shortage = sample_size - current_size
        if verbose:
            print(f"   Short by {shortage:,} samples, filling with highest-weight remaining...")
        
        # Get samples not already included
        used_indices = set(stratified_sample.index) if hasattr(stratified_sample, 'index') else set()
        remaining_df = filtered_df[~filtered_df.index.isin(used_indices)]
        
        if len(remaining_df) > 0:
            # Sort by weight and take top samples
            top_remaining = remaining_df.nlargest(shortage, 'edge_weight')
            stratified_sample = pd.concat([stratified_sample, top_remaining], ignore_index=True)
    
    # Step 8: Final shuffle to mix relations
    stratified_sample = stratified_sample.sample(frac=1.0, random_state=42).reset_index(drop=True)
    
    # Step 9: Validation and statistics
    final_size = len(stratified_sample)
    final_relation_counts = stratified_sample['relation_type'].value_counts()
    final_relation_proportions = final_relation_counts / final_size
    
    print(f"\n✅ Stratified sample created!")
    print(f"   Final size: {final_size:,} triples")
    print(f"   Weight range: {stratified_sample['edge_weight'].min():.3f} - {stratified_sample['edge_weight'].max():.3f}")
    print(f"   Mean weight: {stratified_sample['edge_weight'].mean():.3f}")
    
    if verbose:
        print(f"\n📊 Final relation distribution (top 10):")
        for relation, prop in final_relation_proportions.head(10).items():
            count = final_relation_counts[relation]
            original_prop = relation_proportions.get(relation, 0)
            print(f"   {relation}: {count:,} ({prop*100:.1f}% vs {original_prop*100:.1f}% orig)")
    
    return stratified_sample

#### Phase 2A. 
***
Created stratified sampling for agent seeded nodes

In [7]:
resample_data = True

if resample_data:
    # Apply the stratified sampling to your cleaned data
    print("🎲 Creating stratified weighted sample for agent initialization...")

    stratified_seed_data = create_stratified_weighted_sample(
        df=cleaned_english_triples,
        sample_size=5000,
        min_weight_threshold=0.5,  # Only include relations with decent confidence
        verbose=True
    )

    # Show sample characteristics
    print(f"\n🔍 Sample characteristics:")
    print(f"Weight distribution:")
    print(stratified_seed_data['edge_weight'].describe())

    print(f"\nTop concept pairs by weight:")
    top_weighted = stratified_seed_data.nlargest(5, 'edge_weight')
    for _, row in top_weighted.iterrows():
        print(f"   {row['start_concept']} --{row['relation_type']}--> {row['end_concept']} (weight: {row['edge_weight']:.3f})")
    # Save the stratified sample
    stratified_output_path = os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_stratified_seed_5k.parquet.gzip')
    stratified_seed_data.to_parquet(stratified_output_path, index=False, compression='gzip')
    print(f"\n💾 Stratified seed data saved to: {stratified_output_path}")
else:
    # Load the stratified sample
    stratified_seed_data = pd.read_parquet(
        os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_full_stratified.parquet.gzip')
    )
    print(f"Loaded {len(stratified_seed_data):,} triples from stratified sample.")

🎲 Creating stratified weighted sample for agent initialization...
🎯 Creating stratified weighted sample of 5,000 triples...
   Filtering triples with weight >= 0.5
   Kept 1,477,248 of 1,655,522 triples (89.2%)

📊 Original relation distribution (top 10):
   RelatedTo: 417,772 (28.3%)
   DerivedFrom: 324,167 (21.9%)
   FormOf: 294,073 (19.9%)
   Synonym: 107,359 (7.3%)
   IsA: 65,328 (4.4%)
   UsedFor: 39,470 (2.7%)
   AtLocation: 27,708 (1.9%)
   HasSubevent: 25,238 (1.7%)
   HasPrerequisite: 22,710 (1.5%)
   CapableOf: 22,677 (1.5%)

⚖️  Sorting by weight within each relation...

🎯 Target samples per relation:
   RelatedTo: 1,414
   DerivedFrom: 1,097
   FormOf: 995
   Synonym: 363
   IsA: 221
   UsedFor: 133
   AtLocation: 93
   HasSubevent: 85
   HasPrerequisite: 76
   CapableOf: 76
   Total targeted: 4,983

🔄 Sampling from each relation group...


Sampling relations: 100%|██████████| 47/47 [00:02<00:00, 22.54it/s]


   Short by 17 samples, filling with highest-weight remaining...

✅ Stratified sample created!
   Final size: 5,000 triples
   Weight range: 0.500 - 22.891
   Mean weight: 4.063

📊 Final relation distribution (top 10):
   RelatedTo: 1,424 (28.5% vs 28.3% orig)
   DerivedFrom: 1,097 (21.9% vs 21.9% orig)
   FormOf: 995 (19.9% vs 19.9% orig)
   Synonym: 363 (7.3% vs 7.3% orig)
   IsA: 226 (4.5% vs 4.4% orig)
   UsedFor: 133 (2.7% vs 2.7% orig)
   AtLocation: 93 (1.9% vs 1.9% orig)
   HasSubevent: 85 (1.7% vs 1.7% orig)
   CapableOf: 77 (1.5% vs 1.5% orig)
   HasPrerequisite: 76 (1.5% vs 1.5% orig)

🔍 Sample characteristics:
Weight distribution:
count    5000.000000
mean        4.062885
std         2.422774
min         0.500000
25%         2.000000
50%         2.828000
75%         5.759000
max        22.891000
Name: edge_weight, dtype: float64

Top concept pairs by weight:
   baseball --IsA--> sport (weight: 22.891)
   baseball --IsA--> sport (weight: 22.891)
   yo_yo --IsA--> toy (weight

# Phase 3 
***
Agent Initialization

In [8]:
class KnowledgeGraphAgent:
    """
    MVP Knowledge Graph Agent with self-validation and extensible structure
    """
    
    def __init__(self, validate_on_add=True, verbose=True):
        self.graph = nx.MultiDiGraph()  # Allows multiple edges between same nodes
        self.validate_on_add = validate_on_add
        self.verbose = verbose
        self.validation_stats = {
            'total_attempted': 0,
            'successful_adds': 0,
            'duplicates_rejected': 0,
            'contradictions_found': 0,
            'validation_errors': 0
        }
        
        # Contradiction rules - relations that shouldn't coexist
        self.contradiction_rules = {
            'Antonym': ['Synonym', 'RelatedTo'],
            'Synonym': ['Antonym'],
            'Causes': ['Prevents'],
            'Prevents': ['Causes']
        }
        
        print("🧠 Knowledge Graph Agent initialized!")
        print(f"   Validation: {'ON' if validate_on_add else 'OFF'}")
        print(f"   Verbose mode: {'ON' if verbose else 'OFF'}")
    
    def clean_conceptnet_data(self, df):
        """
        Properly clean ConceptNet data - fixes the string splitting issue
        """
        print("🧹 Cleaning ConceptNet data...")
        
        def extract_concept(concept_string):
            """Extract clean concept from ConceptNet URI format"""
            if pd.isna(concept_string):
                return None
            
            # ConceptNet format: /c/en/concept_name/part_of_speech
            # We want the concept_name part
            parts = str(concept_string).split('/')
            if len(parts) >= 4 and parts[1] == 'c' and parts[2] == 'en':
                concept = parts[3]
                # Handle underscores and clean up
                concept = concept.replace('_', ' ')
                return concept
            return concept_string
        
        def extract_relation(relation_string):
            """Extract relation type from ConceptNet URI"""
            if pd.isna(relation_string):
                return None
            parts = str(relation_string).split('/')
            if len(parts) >= 3 and parts[1] == 'r':
                return parts[2]
            return relation_string
        
        def extract_weight(weight_string):
            """Extract numerical weight from JSON string"""
            try:
                weight_data = json.loads(weight_string)
                return float(weight_data.get('weight', 1.0))
            except:
                return 1.0
        
        # Apply cleaning functions
        cleaned_df = df.copy()
        
        print("   Extracting concepts and relations...")
        cleaned_df['start_concept'] = df['start'].apply(extract_concept)
        cleaned_df['end_concept'] = df['end'].apply(extract_concept)
        cleaned_df['relation_type'] = df['relation'].apply(extract_relation)
        cleaned_df['edge_weight'] = df['weight'].apply(extract_weight)
        
        # Filter out invalid entries
        initial_count = len(cleaned_df)
        cleaned_df = cleaned_df.dropna(subset=['start_concept', 'end_concept', 'relation_type'])
        final_count = len(cleaned_df)
        
        print(f"   Filtered {initial_count - final_count:,} invalid entries")
        print(f"   Clean dataset: {final_count:,} triples")
        
        return cleaned_df[['start_concept', 'end_concept', 'relation_type', 'edge_weight']]
    
    def validate_triple(self, start, relation, end, weight=1.0):
        """
        Validate a triple before adding to graph
        Returns: (is_valid, reason)
        """
        # Check for duplicates
        if self.graph.has_edge(start, end):
            existing_edges = self.graph[start][end]
            for edge_data in existing_edges.values():
                if edge_data.get('relation') == relation:
                    return False, f"Duplicate: {start} --{relation}--> {end}"
        
        # Check for contradictions
        if relation in self.contradiction_rules:
            contradictory_relations = self.contradiction_rules[relation]
            
            if self.graph.has_edge(start, end):
                for edge_data in self.graph[start][end].values():
                    if edge_data.get('relation') in contradictory_relations:
                        return False, f"Contradiction: {start} already has {edge_data.get('relation')} with {end}"
        
        # Passed all validation checks
        return True, "Valid"
    
    def add_triple(self, start, relation, end, weight=1.0, force=False):
        """
        Add a validated triple to the knowledge graph
        """
        self.validation_stats['total_attempted'] += 1
        
        if not force and self.validate_on_add:
            is_valid, reason = self.validate_triple(start, relation, end, weight)
            
            if not is_valid:
                if "Duplicate" in reason:
                    self.validation_stats['duplicates_rejected'] += 1
                elif "Contradiction" in reason:
                    self.validation_stats['contradictions_found'] += 1
                else:
                    self.validation_stats['validation_errors'] += 1
                
                if self.verbose:
                    print(f"❌ Rejected: {reason}")
                return False
        
        # Add the triple to graph
        self.graph.add_edge(start, end, relation=relation, weight=weight)
        self.validation_stats['successful_adds'] += 1
        
        if self.verbose and self.validation_stats['total_attempted'] % 1000 == 0:
            print(f"✅ Added {self.validation_stats['successful_adds']:,} triples so far...")
        
        return True
    
    def bulk_load_triples(self, df, max_triples=None):
        """
        Efficiently load multiple triples with progress tracking
        """
        print(f"📊 Loading triples into knowledge graph...")
        
        if max_triples:
            df = df.head(max_triples)
            print(f"   Limited to first {max_triples:,} triples")
        
        total_rows = len(df)
        print(f"   Processing {total_rows:,} triples...")
        
        # Use tqdm for progress bar
        for idx, row in tqdm(df.iterrows(), total=total_rows, desc="Loading triples"):
            self.add_triple(
                start=str(row['start_concept']),
                relation=str(row['relation_type']),
                end=str(row['end_concept']),
                weight=float(row['edge_weight'])
            )
        
        self.print_stats()
    
    def print_stats(self):
        """Print comprehensive statistics about the knowledge graph"""
        print("\n📈 Knowledge Graph Statistics:")
        print("="*50)
        print(f"🔢 Total nodes: {self.graph.number_of_nodes():,}")
        print(f"🔗 Total edges: {self.graph.number_of_edges():,}")
        print(f"📊 Average degree: {np.mean([d for n, d in self.graph.degree()]):,.2f}")
        
        print(f"\n🔍 Validation Results:")
        print(f"   Attempted additions: {self.validation_stats['total_attempted']:,}")
        print(f"   ✅ Successful: {self.validation_stats['successful_adds']:,}")
        print(f"   🔄 Duplicates rejected: {self.validation_stats['duplicates_rejected']:,}")
        print(f"   ⚡ Contradictions found: {self.validation_stats['contradictions_found']:,}")
        print(f"   ❌ Other errors: {self.validation_stats['validation_errors']:,}")
        
        success_rate = (self.validation_stats['successful_adds'] / max(self.validation_stats['total_attempted'], 1)) * 100
        print(f"   📊 Success rate: {success_rate:.2f}%")
        
        # Relation type distribution
        relation_counts = Counter()
        for _, _, data in self.graph.edges(data=True):
            relation_counts[data.get('relation', 'Unknown')] += 1
        
        print(f"\n🏷️  Top 10 Relation Types:")
        for relation, count in relation_counts.most_common(10):
            print(f"   {relation}: {count:,}")
    
    def query_concept(self, concept, max_results=10):
        """
        Query all relations for a given concept
        """
        if concept not in self.graph:
            print(f"❓ Concept '{concept}' not found in graph")
            return []
        
        print(f"🔍 Relations for '{concept}':")
        
        results = []
        
        # Outgoing relations
        for neighbor in list(self.graph.neighbors(concept))[:max_results//2]:
            edge_data = self.graph[concept][neighbor]
            for edge in edge_data.values():
                relation = edge.get('relation', 'Unknown')
                weight = edge.get('weight', 1.0)
                results.append((concept, relation, neighbor, weight, 'outgoing'))
                print(f"   {concept} --{relation}--> {neighbor} (weight: {weight:.2f})")
        
        # Incoming relations
        for predecessor in list(self.graph.predecessors(concept))[:max_results//2]:
            edge_data = self.graph[predecessor][concept]
            for edge in edge_data.values():
                relation = edge.get('relation', 'Unknown')
                weight = edge.get('weight', 1.0)
                results.append((predecessor, relation, concept, weight, 'incoming'))
                print(f"   {predecessor} --{relation}--> {concept} (weight: {weight:.2f})")
        
        return results
    
    def find_path(self, start_concept, end_concept, max_length=3):
        """
        Find connection paths between two concepts
        """
        if start_concept not in self.graph or end_concept not in self.graph:
            print(f"❓ One or both concepts not found in graph")
            return []
        
        try:
            # Find shortest path
            path = nx.shortest_path(self.graph, start_concept, end_concept, weight=None)
            
            print(f"🛤️  Path from '{start_concept}' to '{end_concept}':")
            
            # Print the path with relations
            for i in range(len(path) - 1):
                current = path[i]
                next_node = path[i + 1]
                
                if self.graph.has_edge(current, next_node):
                    edge_data = list(self.graph[current][next_node].values())[0]
                    relation = edge_data.get('relation', 'Unknown')
                    print(f"   {current} --{relation}--> {next_node}")
                
            return path
            
        except nx.NetworkXNoPath:
            print(f"❌ No path found between '{start_concept}' and '{end_concept}'")
            return []

In [9]:
# Initialize agent
agent = KnowledgeGraphAgent(validate_on_add=True, verbose=False)

# Test with sample data (replace with your actual cleaned data)
sample_data = pd.DataFrame({
    'start_concept': ['dog', 'cat', 'dog', 'happy', 'sad'],
    'end_concept': ['animal', 'animal', 'cat', 'emotion', 'emotion'],
    'relation_type': ['IsA', 'IsA', 'RelatedTo', 'IsA', 'IsA'],
    'edge_weight': [1.0, 1.0, 0.8, 1.0, 1.0]
})

print("📝 Loading sample data...")
agent.bulk_load_triples(sample_data)

print("\n🔍 Testing queries...")
agent.query_concept('dog')

print("\n🛤️  Testing path finding...")
agent.find_path('dog', 'emotion')

🧠 Knowledge Graph Agent initialized!
   Validation: ON
   Verbose mode: OFF
📝 Loading sample data...
📊 Loading triples into knowledge graph...
   Processing 5 triples...


Loading triples: 100%|██████████| 5/5 [00:00<00:00, 4989.66it/s]


📈 Knowledge Graph Statistics:
🔢 Total nodes: 6
🔗 Total edges: 5
📊 Average degree: 1.67

🔍 Validation Results:
   Attempted additions: 5
   ✅ Successful: 5
   🔄 Duplicates rejected: 0
   ⚡ Contradictions found: 0
   ❌ Other errors: 0
   📊 Success rate: 100.00%

🏷️  Top 10 Relation Types:
   IsA: 4
   RelatedTo: 1

🔍 Testing queries...
🔍 Relations for 'dog':
   dog --IsA--> animal (weight: 1.00)
   dog --RelatedTo--> cat (weight: 0.80)

🛤️  Testing path finding...
❌ No path found between 'dog' and 'emotion'


[]

#### Phase 3.A 
***
Testing agent interaction

In [10]:
print("\n" + "="*60)
print("🎯 TESTING THE AGENT")
print("="*60)

# Test 1: Query specific concepts
print("\n🔍 Test 1: Querying concept relationships")
test_concepts = ['dog', 'cat', 'animal', 'happy', 'food']

for concept in test_concepts:
    if concept in [str(c).lower() for c in sample_data['start_concept'].unique()]:
        print(f"\n--- Relationships for '{concept}' ---")
        agent.query_concept(concept, max_results=5)
        break

# Test 2: Find paths between concepts
print("\n🛤️  Test 2: Finding concept paths")
# Try to find a path between two concepts
start_concepts = sample_data['start_concept'].value_counts().head(5).index.tolist()
end_concepts = sample_data['end_concept'].value_counts().head(5).index.tolist()

if len(start_concepts) > 0 and len(end_concepts) > 0:
    start_test = str(start_concepts[0])
    end_test = str(end_concepts[0])
    print(f"\nTrying to find path from '{start_test}' to '{end_test}':")
    agent.find_path(start_test, end_test)

# Test 3: Validation effectiveness
print("\n🔬 Test 3: Adding duplicate to test validation")
if len(sample_data) > 0:
    first_row = sample_data.iloc[0]
    print(f"Attempting to add duplicate: {first_row['start_concept']} --{first_row['relation_type']}--> {first_row['end_concept']}")
    
    success = agent.add_triple(
        start=str(first_row['start_concept']),
        relation=str(first_row['relation_type']),
        end=str(first_row['end_concept']),
        weight=float(first_row['edge_weight'])
    )
    
    print(f"Duplicate addition {'succeeded' if success else 'was rejected (as expected)'}")

# Save the processed data for future use
print(f"\n💾 Saving processed data...")
output_path = os.path.join(os.path.dirname(EN_PATH), 'conceptnet_en_processed_for_graph.parquet.gzip')
cleaned_english_triples.to_parquet(output_path, index=False, compression='gzip')
print(f"   Saved to: {output_path}")

print(f"\n🎉 Phase 2 Complete!")
print(f"   Graph loaded with {agent.validation_stats['successful_adds']:,} validated triples")
print(f"   Ready for agent interactions and queries!")


🎯 TESTING THE AGENT

🔍 Test 1: Querying concept relationships

--- Relationships for 'dog' ---
🔍 Relations for 'dog':
   dog --IsA--> animal (weight: 1.00)
   dog --RelatedTo--> cat (weight: 0.80)

🛤️  Test 2: Finding concept paths

Trying to find path from 'dog' to 'animal':
🛤️  Path from 'dog' to 'animal':
   dog --IsA--> animal

🔬 Test 3: Adding duplicate to test validation
Attempting to add duplicate: dog --IsA--> animal
Duplicate addition was rejected (as expected)

💾 Saving processed data...
   Saved to: ../Data/Input\conceptnet_en_processed_for_graph.parquet.gzip

🎉 Phase 2 Complete!
   Graph loaded with 5 validated triples
   Ready for agent interactions and queries!


# Phase 4 
***
Testing if the agent successfully integrates the stratidfied seed

In [11]:
print("🧠 Initializing Knowledge Graph Agent with stratified seed...")
agent = KnowledgeGraphAgent(validate_on_add=True, verbose=False)

# Load the stratified sample
print(f"📊 Loading {len(stratified_seed_data):,} stratified triples...")
agent.bulk_load_triples(stratified_seed_data)

print("\n" + "="*60)
print("🎯 TESTING AGENT WITH STRATIFIED SEED")
print("="*60)

# Test 1: Show relation diversity
print("\n📊 Relation diversity in loaded graph:")
relation_stats = {}
for _, _, data in agent.graph.edges(data=True):
    relation = data.get('relation', 'Unknown')
    if relation not in relation_stats:
        relation_stats[relation] = 0
    relation_stats[relation] += 1

# Show top relations
sorted_relations = sorted(relation_stats.items(), key=lambda x: x[1], reverse=True)
print("Top 10 relations in graph:")
for relation, count in sorted_relations[:10]:
    print(f"   {relation}: {count:,}")

# Test 2: Quality check - show high-weight concepts
print(f"\n⚖️  High-quality relationships (weight > 0.8):")
high_quality_count = 0
for start, end, data in agent.graph.edges(data=True):
    if data.get('weight', 0) > 0.8:
        relation = data.get('relation', 'Unknown')
        weight = data.get('weight', 0)
        print(f"   {start} --{relation}--> {end} (weight: {weight:.3f})")
        high_quality_count += 1
        if high_quality_count >= 10:  # Limit output
            break

print(f"Total high-quality relationships: {high_quality_count:,}")

# Test 3: Concept connectivity analysis
print(f"\n🕸️  Connectivity analysis:")
node_degrees = dict(agent.graph.degree())
top_connected = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:10]

print("Most connected concepts:")
for concept, degree in top_connected:
    print(f"   {concept}: {degree} connections")

# Test 4: Sample queries on well-connected concepts
print(f"\n🔍 Testing queries on top concepts:")
for concept, degree in top_connected[:3]:
    print(f"\n--- Relationships for '{concept}' (degree: {degree}) ---")
    agent.query_concept(concept, max_results=5)

print(f"\n🎉 Agent successfully initialized with high-quality stratified seed!")
print(f"   Ready for knowledge graph reasoning and expansion!")

🧠 Initializing Knowledge Graph Agent with stratified seed...
🧠 Knowledge Graph Agent initialized!
   Validation: ON
   Verbose mode: OFF
📊 Loading 5,000 stratified triples...
📊 Loading triples into knowledge graph...
   Processing 5,000 triples...


Loading triples: 100%|██████████| 5000/5000 [00:00<00:00, 45871.47it/s]


📈 Knowledge Graph Statistics:
🔢 Total nodes: 4,640
🔗 Total edges: 4,956
📊 Average degree: 2.14

🔍 Validation Results:
   Attempted additions: 5,000
   ✅ Successful: 4,956
   🔄 Duplicates rejected: 42
   ⚡ Contradictions found: 2
   ❌ Other errors: 0
   📊 Success rate: 99.12%

🏷️  Top 10 Relation Types:
   RelatedTo: 1,414
   DerivedFrom: 1,097
   FormOf: 991
   Synonym: 345
   IsA: 221
   UsedFor: 133
   AtLocation: 93
   HasSubevent: 85
   HasPrerequisite: 76
   CapableOf: 76

🎯 TESTING AGENT WITH STRATIFIED SEED

📊 Relation diversity in loaded graph:
Top 10 relations in graph:
   RelatedTo: 1,414
   DerivedFrom: 1,097
   FormOf: 991
   Synonym: 345
   IsA: 221
   UsedFor: 133
   AtLocation: 93
   HasSubevent: 85
   HasPrerequisite: 76
   CapableOf: 76

⚖️  High-quality relationships (weight > 0.8):
   antichrist --DerivedFrom--> christ (weight: 2.000)
   n --FormOf--> dixie_cup (weight: 2.828)
   n --FormOf--> pop (weight: 2.828)
   n --FormOf--> babysitting (weight: 2.000)
   n --S

# Phase 5 
***
Test if the Agent memory graph is now interactive

In [12]:
class MVPKnowledgeAgent:
    """
    Minimum Viable Product - Interactive Knowledge Graph Agent
    Core features: Explore, Reason, Validate, Learn
    """
    
    def __init__(self, base_agent):
        self.graph = base_agent.graph
        self.base_agent = base_agent
        self.reasoning_cache = {}  # Cache for expensive operations
        self.learning_log = []     # Track what the agent learns
        
        print("🚀 MVP Knowledge Agent initialized!")
        print(f"   Base knowledge: {self.graph.number_of_nodes():,} concepts")
        print(f"   Relationships: {self.graph.number_of_edges():,} edges")
    
    def explore_concept(self, concept, depth=2, max_results=20):
        """
        Core MVP Feature 1: Deep concept exploration
        Shows not just direct relationships, but relationships of relationships
        """
        print(f"🔍 EXPLORING: '{concept}' (depth: {depth})")
        print("="*50)
        
        if concept not in self.graph:
            # Try fuzzy matching
            similar = self._find_similar_concepts(concept)
            if similar:
                print(f"❓ '{concept}' not found. Did you mean: {', '.join(similar[:3])}?")
                return None
            else:
                print(f"❌ '{concept}' not found in knowledge base")
                return None
        
        exploration_results = {
            'target_concept': concept,
            'direct_relations': [],
            'extended_relations': [],
            'concept_clusters': [],
            'reasoning_paths': []
        }
        
        # Level 1: Direct relationships
        print(f"\n📍 DIRECT RELATIONSHIPS:")
        direct_count = 0
        for neighbor in self.graph.neighbors(concept):
            if direct_count >= max_results // 2:
                break
                
            edge_data = list(self.graph[concept][neighbor].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            exploration_results['direct_relations'].append({
                'from': concept,
                'relation': relation,
                'to': neighbor,
                'weight': weight,
                'type': 'outgoing'
            })
            
            print(f"   {concept} --{relation}--> {neighbor} (w: {weight:.2f})")
            direct_count += 1
        
        # Include incoming relationships
        for predecessor in self.graph.predecessors(concept):
            if direct_count >= max_results // 2:
                break
                
            edge_data = list(self.graph[predecessor][concept].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            exploration_results['direct_relations'].append({
                'from': predecessor,
                'relation': relation,
                'to': concept,
                'weight': weight,
                'type': 'incoming'
            })
            
            print(f"   {predecessor} --{relation}--> {concept} (w: {weight:.2f})")
            direct_count += 1
        
        # Level 2: Extended exploration (relationships of relationships)
        if depth > 1:
            print(f"\n🔄 EXTENDED RELATIONSHIPS (depth 2):")
            extended_concepts = set()
            
            # Get neighbors of neighbors
            for neighbor in list(self.graph.neighbors(concept))[:5]:  # Limit to prevent explosion
                for second_neighbor in list(self.graph.neighbors(neighbor))[:3]:
                    if second_neighbor != concept and second_neighbor not in extended_concepts:
                        extended_concepts.add(second_neighbor)
                        
                        # Get the relation chain
                        edge1 = list(self.graph[concept][neighbor].values())[0]
                        edge2 = list(self.graph[neighbor][second_neighbor].values())[0]
                        
                        relation1 = edge1.get('relation', 'Unknown')
                        relation2 = edge2.get('relation', 'Unknown')
                        
                        exploration_results['extended_relations'].append({
                            'path': [concept, neighbor, second_neighbor],
                            'relations': [relation1, relation2],
                            'reasoning': f"{concept} --{relation1}--> {neighbor} --{relation2}--> {second_neighbor}"
                        })
                        
                        print(f"   {concept} --{relation1}--> {neighbor} --{relation2}--> {second_neighbor}")
                        
                        if len(extended_concepts) >= max_results // 4:
                            break
                if len(extended_concepts) >= max_results // 4:
                    break
        
        return exploration_results
    
    def reason_about_relationship(self, concept1, concept2, max_paths=3):
        """
        Core MVP Feature 2: Reasoning about why two concepts might be related
        """
        print(f"🧠 REASONING: Why might '{concept1}' and '{concept2}' be related?")
        print("="*60)
        
        if concept1 not in self.graph or concept2 not in self.graph:
            missing = [c for c in [concept1, concept2] if c not in self.graph]
            print(f"❌ Concepts not found: {missing}")
            return None
        
        reasoning_results = {
            'concept1': concept1,
            'concept2': concept2,
            'direct_connection': None,
            'reasoning_paths': [],
            'shared_concepts': [],
            'relationship_strength': 0.0
        }
        
        # Check for direct connection
        if self.graph.has_edge(concept1, concept2):
            edge_data = list(self.graph[concept1][concept2].values())[0]
            relation = edge_data.get('relation', 'Unknown')
            weight = edge_data.get('weight', 1.0)
            
            reasoning_results['direct_connection'] = {
                'relation': relation,
                'weight': weight
            }
            
            print(f"✅ DIRECT CONNECTION FOUND:")
            print(f"   {concept1} --{relation}--> {concept2} (weight: {weight:.3f})")
            reasoning_results['relationship_strength'] = weight
        
        # Find indirect reasoning paths
        try:
            paths = list(nx.all_simple_paths(self.graph, concept1, concept2, cutoff=3))[:max_paths]
            
            if paths:
                print(f"\n🛤️  REASONING PATHS FOUND ({len(paths)}):")
                
                for i, path in enumerate(paths, 1):
                    path_relations = []
                    path_weights = []
                    path_description = []
                    
                    for j in range(len(path) - 1):
                        current = path[j]
                        next_node = path[j + 1]
                        
                        if self.graph.has_edge(current, next_node):
                            edge_data = list(self.graph[current][next_node].values())[0]
                            relation = edge_data.get('relation', 'Unknown')
                            weight = edge_data.get('weight', 1.0)
                            
                            path_relations.append(relation)
                            path_weights.append(weight)
                            path_description.append(f"{current} --{relation}--> {next_node}")
                    
                    avg_weight = np.mean(path_weights) if path_weights else 0.0
                    
                    reasoning_results['reasoning_paths'].append({
                        'path': path,
                        'relations': path_relations,
                        'avg_weight': avg_weight,
                        'description': ' → '.join(path_description)
                    })
                    
                    print(f"\n   Path {i} (strength: {avg_weight:.3f}):")
                    for desc in path_description:
                        print(f"      {desc}")
                    
                    # Update relationship strength
                    if avg_weight > reasoning_results['relationship_strength']:
                        reasoning_results['relationship_strength'] = avg_weight
        
        except nx.NetworkXNoPath:
            print(f"❌ No reasoning paths found between '{concept1}' and '{concept2}'")
        
        # Find shared connections (concepts both are related to)
        concept1_neighbors = set(self.graph.neighbors(concept1)) | set(self.graph.predecessors(concept1))
        concept2_neighbors = set(self.graph.neighbors(concept2)) | set(self.graph.predecessors(concept2))
        shared = concept1_neighbors & concept2_neighbors
        
        if shared:
            print(f"\n🤝 SHARED CONNECTIONS ({len(shared)}):")
            for shared_concept in list(shared)[:5]:  # Limit output
                reasoning_results['shared_concepts'].append(shared_concept)
                print(f"   Both relate to: '{shared_concept}'")
        
        # Generate reasoning strength score
        strength_score = reasoning_results['relationship_strength']
        if strength_score > 0.8:
            strength_desc = "STRONG"
        elif strength_score > 0.5:
            strength_desc = "MODERATE"
        elif strength_score > 0.2:
            strength_desc = "WEAK"
        else:
            strength_desc = "MINIMAL"
        
        print(f"\n📊 RELATIONSHIP STRENGTH: {strength_desc} ({strength_score:.3f})")
        
        return reasoning_results
    
    def validate_new_knowledge(self, start_concept, relation, end_concept, confidence=1.0):
        """
        Core MVP Feature 3: Validate if new knowledge makes sense
        """
        print(f"🔬 VALIDATING: {start_concept} --{relation}--> {end_concept}")
        print("="*50)
        
        validation_result = {
            'proposed_triple': (start_concept, relation, end_concept),
            'confidence': confidence,
            'validation_score': 0.0,
            'supporting_evidence': [],
            'contradictions': [],
            'recommendation': 'REJECT'
        }
        
        # Check 1: Do the concepts exist in our knowledge base?
        concept_familiarity = 0
        if start_concept in self.graph:
            concept_familiarity += 0.5
            print(f"✅ Know about '{start_concept}'")
        else:
            print(f"❓ Unknown concept: '{start_concept}'")
        
        if end_concept in self.graph:
            concept_familiarity += 0.5
            print(f"✅ Know about '{end_concept}'")
        else:
            print(f"❓ Unknown concept: '{end_concept}'")
        
        # Check 2: Look for supporting evidence
        if start_concept in self.graph:
            start_relations = [data.get('relation') for _, _, data in self.graph.edges(start_concept, data=True)]
            if relation in start_relations:
                validation_result['supporting_evidence'].append(f"'{start_concept}' commonly uses '{relation}' relation")
                print(f"✅ '{start_concept}' commonly uses '{relation}' relation")
        
        # Check 3: Look for contradictions
        if self.graph.has_edge(start_concept, end_concept):
            existing_relations = [data.get('relation') for data in self.graph[start_concept][end_concept].values()]
            contradictory_relations = self.base_agent.contradiction_rules.get(relation, [])
            
            for existing_rel in existing_relations:
                if existing_rel in contradictory_relations:
                    validation_result['contradictions'].append(f"Contradicts existing '{existing_rel}' relation")
                    print(f"⚠️  Contradicts existing '{existing_rel}' relation")
        
        # Calculate validation score
        score = 0.0
        score += concept_familiarity * 0.3  # 30% for concept familiarity
        score += len(validation_result['supporting_evidence']) * 0.4  # 40% for supporting evidence
        score -= len(validation_result['contradictions']) * 0.5  # Penalty for contradictions
        score += confidence * 0.3  # 30% for stated confidence
        
        validation_result['validation_score'] = max(0.0, min(1.0, score))  # Clamp to [0,1]
        
        # Make recommendation
        if validation_result['validation_score'] > 0.7:
            validation_result['recommendation'] = 'ACCEPT'
            recommendation_color = "✅"
        elif validation_result['validation_score'] > 0.4:
            validation_result['recommendation'] = 'REVIEW'
            recommendation_color = "⚠️ "
        else:
            validation_result['recommendation'] = 'REJECT'
            recommendation_color = "❌"
        
        print(f"\n📊 VALIDATION SCORE: {validation_result['validation_score']:.3f}")
        print(f"{recommendation_color} RECOMMENDATION: {validation_result['recommendation']}")
        
        return validation_result
    
    def learn_new_triple(self, start_concept, relation, end_concept, confidence=1.0, force=False):
        """
        Core MVP Feature 4: Learn and integrate new knowledge
        """
        print(f"🧠 LEARNING: {start_concept} --{relation}--> {end_concept}")
        
        # First validate the knowledge
        validation = self.validate_new_knowledge(start_concept, relation, end_concept, confidence)
        
        should_learn = force or validation['recommendation'] in ['ACCEPT', 'REVIEW']
        
        if should_learn:
            # Add to graph
            success = self.base_agent.add_triple(start_concept, relation, end_concept, confidence)
            
            if success:
                # Log the learning event
                learning_event = {
                    'timestamp': pd.Timestamp.now(),
                    'triple': (start_concept, relation, end_concept),
                    'confidence': confidence,
                    'validation_score': validation['validation_score'],
                    'method': 'forced' if force else 'validated'
                }
                self.learning_log.append(learning_event)
                
                print(f"✅ LEARNED: Successfully integrated new knowledge")
                print(f"   Knowledge base now has {self.graph.number_of_edges():,} relationships")
                return True
            else:
                print(f"❌ FAILED: Could not integrate (duplicate or error)")
                return False
        else:
            print(f"❌ REJECTED: Validation score too low ({validation['validation_score']:.3f})")
            return False
    
    def _find_similar_concepts(self, concept, threshold=0.7):
        """Helper: Find concepts similar to the input (simple string matching)"""
        concept_lower = concept.lower()
        similar = []
        
        for node in self.graph.nodes():
            node_lower = str(node).lower()
            if concept_lower in node_lower or node_lower in concept_lower:
                similar.append(str(node))
            if len(similar) >= 5:
                break
        
        return similar
    
    def interactive_session(self):
        """
        Core MVP Feature 5: Interactive exploration session
        """
        print("🎮 Starting Interactive Knowledge Exploration Session!")
        print("Commands: explore <concept>, reason <concept1> <concept2>, validate <start> <relation> <end>, learn <start> <relation> <end>, quit")
        print("="*80)
        
        while True:
            try:
                user_input = input("\n🤖 mvp_agent> ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("👋 Goodbye! Thanks for exploring knowledge with me!")
                    break
                
                parts = user_input.split()
                if not parts:
                    continue
                
                command = parts[0].lower()
                
                if command == 'explore' and len(parts) >= 2:
                    concept = ' '.join(parts[1:])
                    self.explore_concept(concept)
                
                elif command == 'reason' and len(parts) >= 3:
                    concept1 = parts[1]
                    concept2 = ' '.join(parts[2:])
                    self.reason_about_relationship(concept1, concept2)
                
                elif command == 'validate' and len(parts) >= 4:
                    start = parts[1]
                    relation = parts[2]
                    end = ' '.join(parts[3:])
                    self.validate_new_knowledge(start, relation, end)
                
                elif command == 'learn' and len(parts) >= 4:
                    start = parts[1]
                    relation = parts[2]
                    end = ' '.join(parts[3:])
                    self.learn_new_triple(start, relation, end)
                
                else:
                    print("❓ Unknown command. Try: explore <concept>, reason <concept1> <concept2>, validate/learn <start> <relation> <end>, quit")
            
            except KeyboardInterrupt:
                print("\n👋 Session interrupted. Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")

# Initialize the MVP Agent
print("🚀 Initializing MVP Knowledge Agent...")
mvp_agent = MVPKnowledgeAgent(agent)

print("\n" + "="*60)
print("🎯 MVP AGENT READY - TESTING CORE FEATURES")
print("="*60)

# Test the core features
print("\n1️⃣ Testing Concept Exploration:")
mvp_agent.explore_concept('dog', depth=2)

print("\n2️⃣ Testing Relationship Reasoning:")
mvp_agent.reason_about_relationship('dog', 'animal')

print("\n3️⃣ Testing Knowledge Validation:")
mvp_agent.validate_new_knowledge('dog', 'IsA', 'mammal', confidence=0.9)

print("\n4️⃣ Testing Learning:")
mvp_agent.learn_new_triple('dog', 'CapableOf', 'barking', confidence=0.95)

print(f"\n🎉 MVP Agent is ready! All core features working.")
print(f"📊 Current knowledge: {mvp_agent.graph.number_of_nodes():,} concepts, {mvp_agent.graph.number_of_edges():,} relationships")
print(f"🧠 Learning events: {len(mvp_agent.learning_log)}")

🚀 Initializing MVP Knowledge Agent...
🚀 MVP Knowledge Agent initialized!
   Base knowledge: 4,640 concepts
   Relationships: 4,956 edges

🎯 MVP AGENT READY - TESTING CORE FEATURES

1️⃣ Testing Concept Exploration:
🔍 EXPLORING: 'dog' (depth: 2)

📍 DIRECT RELATIONSHIPS:
   dog --IsA--> canine (w: 4.90)
   dog --CapableOf--> guard_house (w: 10.39)
   dog --CapableOf--> bark (w: 16.00)
   dog --AtLocation--> kennel (w: 9.38)
   dog --Antonym--> cat (w: 3.69)
   dog --Desires--> petted (w: 4.90)
   dog --IsA--> mammal (w: 5.29)
   dog --CapableOf--> run (w: 6.00)
   dog --IsA--> loyal_friend (w: 6.63)
   dog --IsA--> pet (w: 6.00)

🔄 EXTENDED RELATIONSHIPS (depth 2):
   dog --CapableOf--> bark --PartOf--> tree
   dog --Antonym--> cat --CapableOf--> hunt_mice
   dog --Antonym--> cat --RelatedTo--> feline
   dog --Antonym--> cat --AtLocation--> lap

2️⃣ Testing Relationship Reasoning:
🧠 REASONING: Why might 'dog' and 'animal' be related?
✅ DIRECT CONNECTION FOUND:
   dog --RelatedTo--> animal

# Phase 6: Full learning loop

In [13]:
class AdaptiveTrainingLoop:
    """
    Self-improving training loop that processes the entire dataset until convergence
    Uses adaptive batching, quality thresholds, and convergence detection
    """
    
    def __init__(self, mvp_agent, full_dataset, config=None):
        self.agent = mvp_agent
        self.full_dataset = full_dataset.copy()
        self.training_history = []
        self.convergence_metrics = deque(maxlen=10)  # Rolling window for convergence
        
        # Training configuration with speed and verbosity controls
        self.config = config or {
            # Core training parameters
            'initial_batch_size': 1000,
            'max_batch_size': 10000,
            'min_batch_size': 100,
            'quality_threshold_start': 0.3,
            'quality_threshold_end': 0.7,
            'convergence_patience': 3,
            'max_epochs': 20,
            'validation_sample_size': 500,
            'adaptive_threshold': True,
            'learning_rate_decay': 0.95,
            
            # Speed controls
            'test_mode': False,              # Fast testing mode
            'test_sample_size': 10000,       # Max triples in test mode
            'skip_intelligent_batching': False,  # Use simple batching for speed
            'fast_validation': False,        # Skip expensive validation steps
            'vectorized_processing': True,   # Use pandas vectorization where possible
            
            # Verbosity controls  
            'verbose_level': 1,              # 0=quiet, 1=normal, 2=detailed, 3=debug
            'print_frequency': 100,          # Print every N triples processed
            'batch_progress': True,          # Show batch-level progress
            'real_time_stats': True,         # Print stats during processing
            'checkpoint_frequency': 1000     # Print checkpoint every N triples
        }
        
        self.current_epoch = 0
        self.current_batch_size = self.config['initial_batch_size']
        self.current_quality_threshold = self.config['quality_threshold_start']
        self.processed_count = 0
        self.last_checkpoint_time = time.time()
        
        # Apply speed optimizations if in test mode
        if self.config['test_mode']:
            print("⚡ FAST TEST MODE ENABLED")
            if len(self.full_dataset) > self.config['test_sample_size']:
                self.full_dataset = self.full_dataset.sample(
                    n=self.config['test_sample_size'], 
                    random_state=42
                ).reset_index(drop=True)
                print(f"   📊 Dataset limited to {len(self.full_dataset):,} triples")
            
            # Speed up other parameters
            self.config['max_epochs'] = min(3, self.config['max_epochs'])
            self.config['convergence_patience'] = 2
            print(f"   🏃 Max epochs reduced to {self.config['max_epochs']}")
        
        self._print(f"🎯 Adaptive Training Loop Initialized!", level=1)
        self._print(f"   Dataset size: {len(self.full_dataset):,} triples", level=1)
        self._print(f"   Starting batch size: {self.current_batch_size:,}", level=1)
        self._print(f"   Quality threshold: {self.current_quality_threshold:.3f}", level=1)
        self._print(f"   Verbosity level: {self.config['verbose_level']}", level=2)
    
    def _print(self, message, level=1):
        """Controlled printing based on verbosity level"""
        if self.config['verbose_level'] >= level:
            print(message)
    
    def calculate_graph_quality_metrics(self):
        """Calculate comprehensive quality metrics for the current graph state"""
        graph = self.agent.graph
        
        metrics = {
            'total_nodes': graph.number_of_nodes(),
            'total_edges': graph.number_of_edges(),
            'density': nx.density(graph),
            'avg_degree': np.mean([d for n, d in graph.degree()]) if graph.number_of_nodes() > 0 else 0,
            'connected_components': nx.number_weakly_connected_components(graph),
            'avg_weight': 0,
            'high_quality_edges': 0,
            'relation_diversity': 0
        }
        
        # Weight-based metrics
        weights = [data.get('weight', 1.0) for _, _, data in graph.edges(data=True)]
        if weights:
            metrics['avg_weight'] = np.mean(weights)
            metrics['high_quality_edges'] = sum(1 for w in weights if w > 0.7)
        
        # Relation diversity
        relations = [data.get('relation', 'Unknown') for _, _, data in graph.edges(data=True)]
        metrics['relation_diversity'] = len(set(relations))
        
        # Graph connectivity score (higher is better)
        if metrics['total_nodes'] > 0:
            metrics['connectivity_score'] = (metrics['avg_degree'] * metrics['density'] * 
                                           (1 / max(1, metrics['connected_components'])))
        else:
            metrics['connectivity_score'] = 0
        
        return metrics
    
    def create_intelligent_batch(self, remaining_data, batch_size):
        """
        Create an intelligent batch prioritizing:
        1. High-weight triples
        2. Concepts already in the graph (for better connectivity)
        3. Diverse relation types
        4. Novel concepts (for expansion)
        """
        if len(remaining_data) <= batch_size:
            return remaining_data.copy(), pd.DataFrame()
        
        # Fast mode: skip intelligent batching for speed
        if self.config['skip_intelligent_batching']:
            self._print(f"   📊 Creating simple batch of {batch_size:,} (fast mode)", level=2)
            batch = remaining_data.head(batch_size)
            remaining = remaining_data.tail(len(remaining_data) - batch_size)
            return batch, remaining
        
        self._print(f"   📊 Creating intelligent batch of {batch_size:,} from {len(remaining_data):,} remaining...", level=2)
        
        # Score each triple for training value (vectorized for speed)
        if self.config['vectorized_processing']:
            scores = self._calculate_batch_scores_vectorized(remaining_data)
        else:
            scores = self._calculate_batch_scores_iterative(remaining_data)
        
        # Add scores and sort
        remaining_data = remaining_data.copy()
        remaining_data['training_score'] = scores
        remaining_data = remaining_data.sort_values('training_score', ascending=False)
        
        # Take top-scored triples for batch
        batch = remaining_data.head(batch_size).drop('training_score', axis=1)
        remaining = remaining_data.tail(len(remaining_data) - batch_size).drop('training_score', axis=1)
        
        return batch, remaining
    
    def _calculate_batch_scores_vectorized(self, data):
        """Vectorized batch scoring for speed"""
        existing_concepts = set(self.agent.graph.nodes())
        
        # Weight component (40%)
        scores = data['edge_weight'] * 0.4
        
        # Connectivity component (30%) - vectorized
        start_known = data['start_concept'].isin(existing_concepts)
        end_known = data['end_concept'].isin(existing_concepts)
        
        connectivity_scores = np.where(
            start_known & end_known, 0.3,  # Both known
            np.where(start_known | end_known, 0.2, 0.1)  # One known or none
        )
        scores += connectivity_scores
        
        # Relation diversity (20%) - simplified for speed
        common_relations = {'IsA', 'RelatedTo', 'PartOf', 'UsedFor', 'CapableOf'}
        relation_scores = data['relation_type'].isin(common_relations).astype(float) * 0.2
        scores += relation_scores
        
        # Simple validation prediction (10%)
        scores += 0.1  # Simplified
        
        return scores.values
    
    def _calculate_batch_scores_iterative(self, remaining_data):
        """Original iterative scoring (slower but more accurate)"""
        scores = []
        existing_concepts = set(self.agent.graph.nodes())
        
        for idx, row in remaining_data.iterrows():
            score = 0
            
            # Weight component (40%)
            score += row['edge_weight'] * 0.4
            
            # Connectivity component (30%)
            start_known = row['start_concept'] in existing_concepts
            end_known = row['end_concept'] in existing_concepts
            
            if start_known and end_known:
                score += 0.3
            elif start_known or end_known:
                score += 0.2
            else:
                score += 0.1
            
            # Relation diversity component (20%)
            current_relations = [data.get('relation') for _, _, data in self.agent.graph.edges(data=True)]
            relation_counts = pd.Series(current_relations).value_counts()
            current_relation = row['relation_type']
            
            if current_relation not in relation_counts.index:
                score += 0.2
            else:
                relation_freq = relation_counts[current_relation] / len(current_relations)
                score += 0.2 * (1 - relation_freq)
            
            # Validation prediction component (10%)
            common_relations = ['IsA', 'RelatedTo', 'PartOf', 'UsedFor', 'CapableOf']
            if current_relation in common_relations:
                score += 0.1
            
            scores.append(score)
        
        return scores
    
    def validate_batch_quality(self, batch_sample_size=100):
        """
        Validate a sample of the remaining data to estimate quality
        Used for adaptive threshold adjustment
        """
        if len(self.full_dataset) <= batch_sample_size:
            sample = self.full_dataset
        else:
            sample = self.full_dataset.sample(n=batch_sample_size, random_state=42)
        
        validation_scores = []
        
        for _, row in sample.iterrows():
            validation = self.agent.validate_new_knowledge(
                row['start_concept'], 
                row['relation_type'], 
                row['end_concept'], 
                row['edge_weight']
            )
            validation_scores.append(validation['validation_score'])
        
        return {
            'mean_quality': np.mean(validation_scores),
            'std_quality': np.std(validation_scores),
            'high_quality_ratio': sum(1 for s in validation_scores if s > 0.7) / len(validation_scores)
        }
    
    def process_batch(self, batch):
        """Process a batch of triples with detailed tracking and verbose output"""
        batch_stats = {
            'attempted': len(batch),
            'accepted': 0,
            'rejected_duplicate': 0,
            'rejected_validation': 0,
            'rejected_contradiction': 0,
            'avg_validation_score': 0,
            'processing_time': 0
        }
        
        validation_scores = []
        start_time = time.time()
        
        self._print(f"   🔄 Processing batch of {len(batch):,} triples...", level=1)
        
        # Choose progress bar or simple counter based on verbosity
        if self.config['batch_progress'] and self.config['verbose_level'] >= 2:
            iterator = tqdm(batch.iterrows(), total=len(batch), desc="Processing batch", leave=False)
        else:
            iterator = batch.iterrows()
        
        for i, (_, row) in enumerate(iterator):
            # Print real-time progress
            if (self.config['real_time_stats'] and 
                self.config['verbose_level'] >= 3 and 
                (i + 1) % self.config['print_frequency'] == 0):
                current_rate = batch_stats['accepted'] / max(1, i + 1)
                self._print(f"      Progress: {i+1:,}/{len(batch):,} "
                          f"(acceptance: {current_rate:.3f})", level=3)
            
            # Fast validation mode
            if self.config['fast_validation']:
                # Simple validation - just check weight and existing relations
                validation_score = row['edge_weight']
                if self.agent.graph.has_edge(row['start_concept'], row['end_concept']):
                    validation_score *= 0.5  # Penalize duplicates
            else:
                # Full validation
                validation = self.agent.validate_new_knowledge(
                    row['start_concept'], 
                    row['relation_type'], 
                    row['end_concept'], 
                    row['edge_weight']
                )
                validation_score = validation['validation_score']
            
            validation_scores.append(validation_score)
            
            # Decide whether to learn based on current quality threshold
            if validation_score >= self.current_quality_threshold:
                success = self.agent.learn_new_triple(
                    row['start_concept'], 
                    row['relation_type'], 
                    row['end_concept'], 
                    row['edge_weight'],
                    force=False
                )
                
                if success:
                    batch_stats['accepted'] += 1
                    self.processed_count += 1
                    
                    # Checkpoint reporting
                    if (self.config['real_time_stats'] and 
                        self.processed_count % self.config['checkpoint_frequency'] == 0):
                        elapsed = time.time() - self.last_checkpoint_time
                        rate = self.config['checkpoint_frequency'] / elapsed
                        total_edges = self.agent.graph.number_of_edges()
                        self._print(f"    🏁 CHECKPOINT: {self.processed_count:,} processed "
                                  f"(rate: {rate:.1f}/sec, total edges: {total_edges:,})", level=2)
                        self.last_checkpoint_time = time.time()
                else:
                    batch_stats['rejected_duplicate'] += 1
            else:
                batch_stats['rejected_validation'] += 1
        
        batch_stats['avg_validation_score'] = np.mean(validation_scores)
        batch_stats['processing_time'] = time.time() - start_time
        
        # Detailed batch summary
        acceptance_rate = batch_stats['accepted'] / max(1, batch_stats['attempted'])
        processing_rate = batch_stats['attempted'] / batch_stats['processing_time']
        
        self._print(f"      ✅ Batch complete: {batch_stats['accepted']}/{batch_stats['attempted']} accepted "
                  f"({acceptance_rate:.3f})", level=1)
        self._print(f"      ⚡ Processing rate: {processing_rate:.1f} triples/sec", level=2)
        self._print(f"      📊 Avg validation score: {batch_stats['avg_validation_score']:.3f}", level=2)
        
        return batch_stats
    
    def update_training_parameters(self, epoch_stats):
        """
        Adaptively update training parameters based on performance
        """
        self._print(f"   🎛️  Updating training parameters...", level=2)
        
        # Adaptive batch size
        acceptance_rate = epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted'])
        
        if acceptance_rate > 0.8:  # High acceptance, increase batch size
            self.current_batch_size = min(
                self.config['max_batch_size'],
                int(self.current_batch_size * 1.2)
            )
        elif acceptance_rate < 0.3:  # Low acceptance, decrease batch size for quality
            self.current_batch_size = max(
                self.config['min_batch_size'],
                int(self.current_batch_size * 0.8)
            )
        
        # Adaptive quality threshold
        if self.config['adaptive_threshold']:
            # Gradually increase threshold as graph improves
            progress = self.current_epoch / self.config['max_epochs']
            self.current_quality_threshold = (
                self.config['quality_threshold_start'] + 
                progress * (self.config['quality_threshold_end'] - self.config['quality_threshold_start'])
            )
        
        self._print(f"      Batch size: {self.current_batch_size:,}", level=1)
        self._print(f"      Quality threshold: {self.current_quality_threshold:.3f}", level=1)
    
    def check_convergence(self, epoch_stats):
        """
        Check if the training has converged (no more meaningful improvements)
        """
        # Add current metrics to rolling window
        improvement_score = epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted'])
        self.convergence_metrics.append(improvement_score)
        
        if len(self.convergence_metrics) < self.config['convergence_patience']:
            return False
        
        # Check if improvement has plateaued
        recent_improvements = list(self.convergence_metrics)
        trend = np.polyfit(range(len(recent_improvements)), recent_improvements, 1)[0]
        
        # Convergence criteria
        avg_recent_improvement = np.mean(recent_improvements)
        improvement_std = np.std(recent_improvements)
        
        # More aggressive convergence in test mode
        convergence_threshold = (0.02 if self.config['test_mode'] 
                               else self.config.get('convergence_threshold', 0.05))
        
        has_converged = (
            avg_recent_improvement < convergence_threshold or
            (trend <= 0 and improvement_std < 0.02)
        )
        
        if has_converged:
            self._print(f"   🎯 CONVERGENCE DETECTED:", level=1)
            self._print(f"      Recent improvement: {avg_recent_improvement:.4f}", level=1)
            self._print(f"      Trend: {trend:.6f}", level=2)
            self._print(f"      Stability: {improvement_std:.4f}", level=2)
        
        return has_converged
    
    def train_until_convergence(self):
        """
        Main training loop - processes entire dataset until convergence
        Now with configurable speed and verbosity
        """
        self._print("🚀 STARTING ADAPTIVE TRAINING LOOP", level=1)
        self._print("="*60, level=1)
        
        remaining_data = self.full_dataset.copy()
        training_start_time = time.time()
        
        # Pre-training summary
        self._print(f"📊 Training Configuration:", level=1)
        self._print(f"   Dataset size: {len(remaining_data):,} triples", level=1)
        self._print(f"   Test mode: {'ON' if self.config['test_mode'] else 'OFF'}", level=1)
        self._print(f"   Fast validation: {'ON' if self.config['fast_validation'] else 'OFF'}", level=1)
        self._print(f"   Intelligent batching: {'OFF' if self.config['skip_intelligent_batching'] else 'ON'}", level=1)
        self._print(f"   Max epochs: {self.config['max_epochs']}", level=1)
        
        while self.current_epoch < self.config['max_epochs']:
            self.current_epoch += 1
            epoch_start_time = time.time()
            
            self._print(f"\n📅 EPOCH {self.current_epoch}/{self.config['max_epochs']}", level=1)
            self._print("-" * 40, level=1)
            
            # Calculate pre-epoch metrics
            pre_metrics = self.calculate_graph_quality_metrics()
            self._print(f"   Graph state: {pre_metrics['total_nodes']:,} nodes, {pre_metrics['total_edges']:,} edges", level=1)
            self._print(f"   Connectivity score: {pre_metrics['connectivity_score']:.3f}", level=2)
            self._print(f"   Average weight: {pre_metrics['avg_weight']:.3f}", level=2)
            
            # Process data in batches
            epoch_stats = {
                'total_attempted': 0,
                'total_accepted': 0,
                'total_rejected': 0,
                'batches_processed': 0,
                'avg_validation_score': 0
            }
            
            validation_scores = []
            batch_times = []
            
            while len(remaining_data) > 0:
                batch_start_time = time.time()
                
                # Create intelligent batch
                batch, remaining_data = self.create_intelligent_batch(
                    remaining_data, 
                    self.current_batch_size
                )
                
                # Process batch
                batch_stats = self.process_batch(batch)
                
                # Track timing
                batch_time = time.time() - batch_start_time
                batch_times.append(batch_time)
                
                # Update epoch statistics
                epoch_stats['total_attempted'] += batch_stats['attempted']
                epoch_stats['total_accepted'] += batch_stats['accepted']
                epoch_stats['total_rejected'] += (batch_stats['rejected_duplicate'] + 
                                                 batch_stats['rejected_validation'] + 
                                                 batch_stats['rejected_contradiction'])
                epoch_stats['batches_processed'] += 1
                validation_scores.append(batch_stats['avg_validation_score'])
                
                # Batch summary
                acceptance_rate = batch_stats['accepted'] / max(1, batch_stats['attempted'])
                remaining_count = len(remaining_data)
                
                self._print(f"      Batch {epoch_stats['batches_processed']}: "
                          f"{batch_stats['accepted']}/{batch_stats['attempted']} accepted "
                          f"({acceptance_rate:.3f}), {remaining_count:,} remaining", level=1)
                
                # Detailed batch info
                self._print(f"         Validation score: {batch_stats['avg_validation_score']:.3f}, "
                          f"Time: {batch_time:.1f}s", level=2)
                
                # Early stopping if acceptance rate is very low
                if acceptance_rate < 0.01 and epoch_stats['batches_processed'] > 3:
                    self._print(f"      ⚠️  Early batch stopping - acceptance rate too low ({acceptance_rate:.4f})", level=1)
                    break
            
            # Calculate epoch-level metrics
            epoch_stats['avg_validation_score'] = np.mean(validation_scores) if validation_scores else 0
            post_metrics = self.calculate_graph_quality_metrics()
            epoch_duration = time.time() - epoch_start_time
            avg_batch_time = np.mean(batch_times) if batch_times else 0
            
            # Calculate improvements
            node_growth = post_metrics['total_nodes'] - pre_metrics['total_nodes']
            edge_growth = post_metrics['total_edges'] - pre_metrics['total_edges']
            
            epoch_summary = {
                'epoch': self.current_epoch,
                'duration': epoch_duration,
                'acceptance_rate': epoch_stats['total_accepted'] / max(1, epoch_stats['total_attempted']),
                'node_growth': node_growth,
                'edge_growth': edge_growth,
                'final_nodes': post_metrics['total_nodes'],
                'final_edges': post_metrics['total_edges'],
                'avg_validation_score': epoch_stats['avg_validation_score'],
                'connectivity_score': post_metrics['connectivity_score'],
                'batches_processed': epoch_stats['batches_processed'],
                'avg_batch_time': avg_batch_time
            }
            
            self.training_history.append(epoch_summary)
            
            # Detailed epoch summary
            self._print(f"\n   📊 EPOCH {self.current_epoch} SUMMARY:", level=1)
            self._print(f"      Duration: {epoch_duration:.1f}s ({epoch_stats['batches_processed']} batches)", level=1)
            self._print(f"      Acceptance rate: {epoch_summary['acceptance_rate']:.3f}", level=1)
            self._print(f"      Growth: +{node_growth:,} nodes, +{edge_growth:,} edges", level=1)
            self._print(f"      Total: {post_metrics['total_nodes']:,} nodes, {post_metrics['total_edges']:,} edges", level=1)
            
            # Performance metrics
            total_processed = epoch_stats['total_attempted']
            processing_rate = total_processed / epoch_duration if epoch_duration > 0 else 0
            self._print(f"      Processing rate: {processing_rate:.1f} triples/sec", level=2)
            self._print(f"      Avg validation score: {epoch_stats['avg_validation_score']:.3f}", level=2)
            self._print(f"      Connectivity improvement: {post_metrics['connectivity_score'] - pre_metrics['connectivity_score']:.3f}", level=2)
            
            # Quality metrics
            quality_ratio = post_metrics['high_quality_edges'] / max(1, post_metrics['total_edges'])
            self._print(f"      High-quality edge ratio: {quality_ratio:.3f}", level=2)
            self._print(f"      Relation diversity: {post_metrics['relation_diversity']}", level=2)
            
            # Update parameters for next epoch
            self.update_training_parameters(epoch_stats)
            
            # Check for convergence
            if self.check_convergence(epoch_stats):
                self._print(f"\n🎯 CONVERGENCE ACHIEVED after {self.current_epoch} epochs!", level=1)
                break
            
            # Early stopping for very poor performance
            if epoch_summary['acceptance_rate'] < 0.005 and self.current_epoch > 1:
                self._print(f"\n⚠️  EARLY STOPPING - acceptance rate too low ({epoch_summary['acceptance_rate']:.4f})", level=1)
                break
            
            # Reset remaining data for next epoch
            remaining_data = self.full_dataset.copy()
            
            # Memory cleanup suggestion for large datasets
            if len(self.full_dataset) > 50000 and self.current_epoch % 2 == 0:
                self._print("      💾 Consider running garbage collection for memory optimization", level=3)
        
        total_training_time = time.time() - training_start_time
        final_metrics = self.calculate_graph_quality_metrics()
        
        # Final summary
        self._print(f"\n🎉 TRAINING COMPLETE!", level=1)
        self._print("="*60, level=1)
        self._print(f"Total training time: {total_training_time/60:.1f} minutes", level=1)
        self._print(f"Epochs completed: {self.current_epoch}", level=1)
        self._print(f"Final graph: {final_metrics['total_nodes']:,} nodes, {final_metrics['total_edges']:,} edges", level=1)
        
        # Performance summary
        total_processed = sum(h['final_edges'] - (self.training_history[0]['final_edges'] if self.training_history else 0) 
                            for h in self.training_history[-1:])
        avg_processing_rate = total_processed / total_training_time if total_training_time > 0 else 0
        self._print(f"Average processing rate: {avg_processing_rate:.1f} edges/sec", level=1)
        
        # Quality summary
        final_quality_ratio = final_metrics['high_quality_edges'] / max(1, final_metrics['total_edges'])
        self._print(f"Final high-quality edge ratio: {final_quality_ratio:.3f}", level=1)
        self._print(f"Final connectivity score: {final_metrics['connectivity_score']:.3f}", level=1)
        self._print(f"Relation diversity: {final_metrics['relation_diversity']} types", level=1)
        
        # Configuration used
        self._print(f"\nConfiguration used:", level=2)
        self._print(f"   Test mode: {'YES' if self.config['test_mode'] else 'NO'}", level=2)
        self._print(f"   Fast validation: {'YES' if self.config['fast_validation'] else 'NO'}", level=2)
        self._print(f"   Intelligent batching: {'NO' if self.config['skip_intelligent_batching'] else 'YES'}", level=2)
        
        return self.training_history
    
    def plot_training_progress(self):
        """Visualize training progress"""
        if not self.training_history:
            print("No training history to plot")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        epochs = [h['epoch'] for h in self.training_history]
        
        # Plot 1: Acceptance rate over time
        acceptance_rates = [h['acceptance_rate'] for h in self.training_history]
        axes[0,0].plot(epochs, acceptance_rates, 'b-o')
        axes[0,0].set_title('Acceptance Rate Over Time')
        axes[0,0].set_xlabel('Epoch')
        axes[0,0].set_ylabel('Acceptance Rate')
        axes[0,0].grid(True)
        
        # Plot 2: Graph growth
        total_edges = [h['final_edges'] for h in self.training_history]
        axes[0,1].plot(epochs, total_edges, 'g-o')
        axes[0,1].set_title('Knowledge Graph Growth')
        axes[0,1].set_xlabel('Epoch')
        axes[0,1].set_ylabel('Total Edges')
        axes[0,1].grid(True)
        
        # Plot 3: Validation scores
        val_scores = [h['avg_validation_score'] for h in self.training_history]
        axes[1,0].plot(epochs, val_scores, 'r-o')
        axes[1,0].set_title('Average Validation Score')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('Validation Score')
        axes[1,0].grid(True)
        
        # Plot 4: Edge growth per epoch
        edge_growth = [h['edge_growth'] for h in self.training_history]
        axes[1,1].bar(epochs, edge_growth, alpha=0.7)
        axes[1,1].set_title('New Edges Per Epoch')
        axes[1,1].set_xlabel('Epoch')
        axes[1,1].set_ylabel('New Edges Added')
        axes[1,1].grid(True)
        
        plt.tight_layout()
        plt.show()


In [14]:
# Usage example - Updated with speed and verbosity controls
print("🎯 Setting up Configurable Training Loop...")

# FAST TEST CONFIGURATION - for development and testing
fast_test_config = {
    # Speed optimizations
    'test_mode': True,                   # Enable fast mode
    'test_sample_size': 5000,           # Only 5K triples for quick testing
    'initial_batch_size': 200,          # Smaller batches
    'max_batch_size': 1000,            # Lower max batch size
    'min_batch_size': 50,
    'skip_intelligent_batching': True,  # Simple batching for speed
    'fast_validation': True,            # Skip expensive validation
    'vectorized_processing': True,      # Use pandas vectorization
    
    # Training parameters
    'quality_threshold_start': 0.4,     # More permissive
    'quality_threshold_end': 0.6,       # Still maintain some quality
    'max_epochs': 3,                    # Just 3 epochs for testing
    'convergence_patience': 2,          # Quick convergence detection
    'adaptive_threshold': False,        # Fixed threshold for simplicity
    
    # Verbosity controls
    'verbose_level': 3,                 # Maximum verbosity
    'print_frequency': 25,              # Print every 25 triples
    'batch_progress': True,             # Show progress bars
    'real_time_stats': True,            # Real-time statistics
    'checkpoint_frequency': 200         # Checkpoint every 200 triples
}
# PRODUCTION CONFIGURATION - for full training runs
production_config = {
   # Full processing
   'test_mode': False,
   'skip_intelligent_batching': False,
   'fast_validation': False,
   'vectorized_processing': True,
   
   # Standard training parameters
   'initial_batch_size': 2000,
   'max_batch_size': 10000,
   'min_batch_size': 100,
   'quality_threshold_start': 0.3,
   'quality_threshold_end': 0.7,
   'max_epochs': 15,
   'convergence_patience': 3,
   'adaptive_threshold': True,
   
   # Moderate verbosity
   'verbose_level': 2,
   'print_frequency': 100,
   'batch_progress': True,
   'real_time_stats': True,
   'checkpoint_frequency': 1000
}

# SILENT FAST MODE - for background processing
silent_fast_config = {
   'test_mode': True,
   'test_sample_size': 10000,
   'initial_batch_size': 500,
   'max_batch_size': 2000,
   'min_batch_size': 100,
   'skip_intelligent_batching': True,
   'fast_validation': True,
   'max_epochs': 5,
   'quality_threshold_start': 0.4,
   'quality_threshold_end': 0.6,
   'convergence_patience': 2,
   'adaptive_threshold': False,
   
   # Minimal verbosity
   'verbose_level': 1,
   'print_frequency': 1000,
   'batch_progress': False,
   'real_time_stats': False,
   'checkpoint_frequency': 2000
}

🎯 Setting up Configurable Training Loop...


In [15]:
# Choose your configuration
print("📋 Available configurations:")
print("   1. fast_test_config - Quick testing with maximum verbosity")
print("   2. production_config - Full training with moderate verbosity") 
print("   3. silent_fast_config - Fast training with minimal output")

📋 Available configurations:
   1. fast_test_config - Quick testing with maximum verbosity
   2. production_config - Full training with moderate verbosity
   3. silent_fast_config - Fast training with minimal output


In [16]:
# Initialize with fast test config (change as needed)
trainer = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples, fast_test_config)

print(f"\n🚀 Ready to train with FAST TEST configuration!")
print(f"⚡ This should take ~2-5 minutes instead of hours")
print(f"🔊 Maximum verbosity - you'll see everything happening")
print(f"🎯 Start training with:")
print(f"   training_history = trainer.train_until_convergence()")
training_history = trainer.train_until_convergence()

⚡ FAST TEST MODE ENABLED
   📊 Dataset limited to 5,000 triples
   🏃 Max epochs reduced to 3
🎯 Adaptive Training Loop Initialized!
   Dataset size: 5,000 triples
   Starting batch size: 200
   Quality threshold: 0.400
   Verbosity level: 3

🚀 Ready to train with FAST TEST configuration!
⚡ This should take ~2-5 minutes instead of hours
🔊 Maximum verbosity - you'll see everything happening
🎯 Start training with:
   training_history = trainer.train_until_convergence()
🚀 STARTING ADAPTIVE TRAINING LOOP
📊 Training Configuration:
   Dataset size: 5,000 triples
   Test mode: ON
   Fast validation: ON
   Intelligent batching: OFF
   Max epochs: 3

📅 EPOCH 1/3
----------------------------------------
   Graph state: 4,641 nodes, 4,957 edges
   Connectivity score: 0.000
   Average weight: 4.031
   📊 Creating simple batch of 200 (fast mode)
   🔄 Processing batch of 200 triples...


Processing batch:   0%|          | 0/200 [00:00<?, ?it/s]

🧠 LEARNING: v --FormOf--> swinck
🔬 VALIDATING: v --FormOf--> swinck
✅ Know about 'v'
❓ Unknown concept: 'swinck'
✅ 'v' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 0.850
✅ RECOMMENDATION: ACCEPT
✅ LEARNED: Successfully integrated new knowledge
   Knowledge base now has 4,958 relationships
🧠 LEARNING: superviolent --DerivedFrom--> violent
🔬 VALIDATING: superviolent --DerivedFrom--> violent
❓ Unknown concept: 'superviolent'
❓ Unknown concept: 'violent'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: purple_sand_tilefish --IsA--> n
🔬 VALIDATING: purple_sand_tilefish --IsA--> n
❓ Unknown concept: 'purple_sand_tilefish'
✅ Know about 'n'

📊 VALIDATION SCORE: 0.450
⚠️  RECOMMENDATION: REVIEW
✅ LEARNED: Successfully integrated new knowledge
   Knowledge base now has 4,959 relationships
🧠 LEARNING: n --FormOf--> febricity
🔬 VALIDATING: n --FormOf--> febricity
✅ Know about 'n'
❓ Unknown concept: 'febricity'
✅ 'n' commonly uses 

Processing batch:  30%|██▉       | 59/200 [00:00<00:00, 578.11it/s]

   Knowledge base now has 4,972 relationships
      Progress: 25/200 (acceptance: 0.600)
🧠 LEARNING: regioisomer --DerivedFrom--> isomer
🔬 VALIDATING: regioisomer --DerivedFrom--> isomer
❓ Unknown concept: 'regioisomer'
❓ Unknown concept: 'isomer'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: confusion --CausesDesire--> contemplate
🔬 VALIDATING: confusion --CausesDesire--> contemplate
❓ Unknown concept: 'confusion'
✅ Know about 'contemplate'

📊 VALIDATION SCORE: 0.998
✅ RECOMMENDATION: ACCEPT
✅ LEARNED: Successfully integrated new knowledge
   Knowledge base now has 4,973 relationships
🧠 LEARNING: n --FormOf--> accessorial_service
🔬 VALIDATING: n --FormOf--> accessorial_service
✅ Know about 'n'
❓ Unknown concept: 'accessorial_service'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 0.850
✅ RECOMMENDATION: ACCEPT
✅ LEARNED: Successfully integrated new knowledge
   Knowledge base now has 4,974 relationships
🧠 LEARN

...

🧠 LEARNING: reception_area --AtLocation--> office_building
🔬 VALIDATING: reception_area --AtLocation--> office_building
❓ Unknown concept: 'reception_area'
❓ Unknown concept: 'office_building'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: v --FormOf--> play_both_sides_against_middle
🔬 VALIDATING: v --FormOf--> play_both_sides_against_middle
✅ Know about 'v'
✅ Know about 'play_both_sides_against_middle'
✅ 'v' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> adrenalized
🔬 VALIDATING: a --RelatedTo--> adrenalized
✅ Know about 'a'
✅ Know about 'adrenalized'
✅ 'a' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> onshore
🔬 VALIDATING: a --RelatedTo--> onshore
✅ Know about 'a'
✅ Know about 'onshore'
✅ 

🧠 LEARNING: r --Synonym--> gullibly
🔬 VALIDATING: r --Synonym--> gullibly
✅ Know about 'r'
✅ Know about 'gullibly'
✅ 'r' commonly uses 'Synonym' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --RelatedTo--> caffeoylquinic_acid
🔬 VALIDATING: n --RelatedTo--> caffeoylquinic_acid
✅ Know about 'n'
✅ Know about 'caffeoylquinic_acid'
✅ 'n' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: pulicaria --IsA--> n
🔬 VALIDATING: pulicaria --IsA--> n
❓ Unknown concept: 'pulicaria'
✅ Know about 'n'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: weet_bix --DerivedFrom--> n
🔬 VALIDATING: weet_bix --DerivedFrom--> n
✅ Know about 'weet_bix'
✅ Know about 'n'
✅ 'weet_bix' commonly uses 'DerivedFrom' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED:

Processing batch:   0%|          | 0/160 [00:00<?, ?it/s]

🧠 LEARNING: nonbirational --DerivedFrom--> birational
🔬 VALIDATING: nonbirational --DerivedFrom--> birational
❓ Unknown concept: 'nonbirational'
❓ Unknown concept: 'birational'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: squintest --DerivedFrom--> squint
🔬 VALIDATING: squintest --DerivedFrom--> squint
❓ Unknown concept: 'squintest'
❓ Unknown concept: 'squint'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
🧠 LEARNING: n --FormOf--> breviary
🔬 VALIDATING: n --FormOf--> breviary
✅ Know about 'n'
✅ Know about 'breviary'
✅ 'n' commonly uses 'FormOf' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: a --RelatedTo--> unprejudiced
🔬 VALIDATING: a --RelatedTo--> unprejudiced
✅ Know about 'a'
✅ Know about 'unprejudiced'
✅ 'a' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEP


🔬 VALIDATING: playing_basketball --HasSubevent--> fool_opponent
❓ Unknown concept: 'playing_basketball'
❓ Unknown concept: 'fool_opponent'

📊 VALIDATION SCORE: 0.300
❌ RECOMMENDATION: REJECT
❌ REJECTED: Validation score too low (0.300)
      Progress: 25/160 (acceptance: 0.000)
🧠 LEARNING: r --RelatedTo--> greenish
🔬 VALIDATING: r --RelatedTo--> greenish
✅ Know about 'r'
✅ Know about 'greenish'
✅ 'r' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --RelatedTo--> cabinetise
🔬 VALIDATING: n --RelatedTo--> cabinetise
✅ Know about 'n'
✅ Know about 'cabinetise'
✅ 'n' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅ RECOMMENDATION: ACCEPT
❌ FAILED: Could not integrate (duplicate or error)
🧠 LEARNING: n --RelatedTo--> playwork
🔬 VALIDATING: n --RelatedTo--> playwork
✅ Know about 'n'
✅ Know about 'playwork'
✅ 'n' commonly uses 'RelatedTo' relation

📊 VALIDATION SCORE: 1.000
✅

In [17]:
# Quick training starter for immediate testing
print(f"\n🏃 QUICK START - uncomment to begin:")
print(f"# training_history = trainer.train_until_convergence()")
print(f"# trainer.plot_training_progress()")  # Show results when done
# Test with 50K triples first
trainer_50k = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples.head(50000), silent_fast_config)
# training_history = trainer_50k.train_until_convergence()


🏃 QUICK START - uncomment to begin:
# training_history = trainer.train_until_convergence()
# trainer.plot_training_progress()
⚡ FAST TEST MODE ENABLED
   📊 Dataset limited to 10,000 triples
   🏃 Max epochs reduced to 3
🎯 Adaptive Training Loop Initialized!
   Dataset size: 10,000 triples
   Starting batch size: 500
   Quality threshold: 0.400


In [18]:
print(f"\n🚀 Ready to train on {len(cleaned_english_triples):,} triples!")
print(f"⚡ This will take several hours but creates a self-improving agent")
print(f"🎯 Uncomment the next line to start training:")
print(f"# training_history = trainer.train_until_convergence()")
# trainer = AdaptiveTrainingLoop(mvp_agent, cleaned_english_triples, production_config)


🚀 Ready to train on 1,655,522 triples!
⚡ This will take several hours but creates a self-improving agent
🎯 Uncomment the next line to start training:
# training_history = trainer.train_until_convergence()


In [19]:
# ULTRA FAST CONFIGURATION - Process everything as fast as possible
ultra_fast_config = {
    # Speed maximization
    'test_mode': False,                  # Process full dataset
    'initial_batch_size': 50000,        # Massive batches
    'max_batch_size': 100000,           # Even bigger if needed
    'min_batch_size': 10000,            # Large minimum
    'max_epochs': 1,                    # Single pass only
    'convergence_patience': 1,          # Stop immediately if low acceptance
    
    # Skip expensive operations
    'skip_intelligent_batching': True,  # No fancy scoring
    'fast_validation': True,            # Minimal validation
    'vectorized_processing': True,      # Use pandas speed
    'adaptive_threshold': False,        # Fixed threshold
    
    # High quality filter (less processing)
    'quality_threshold_start': 0.7,    # Only high-quality triples
    'quality_threshold_end': 0.7,      # Fixed high threshold
    
    # Minimal output (speed killer removal)
    'verbose_level': 1,                 # Bare minimum output
    'batch_progress': False,            # No progress bars
    'real_time_stats': False,           # No real-time updates
    'print_frequency': 100000,          # Print every 100K only
    'checkpoint_frequency': 500000      # Checkpoint every 500K
}

junk_chars = ['n', 'v', 'r', 'o', 't', 's', 'y', 'c', 'e', 'u']
# REmove start and end concepts that are only junk characters
for junk in junk_chars:
    filtered_english_triples = cleaned_english_triples[~cleaned_english_triples['start_concept'].str.contains(junk)]
    filtered_english_triples = filtered_english_triples[~filtered_english_triples['end_concept'].str.contains(junk)]
    
print(f'Size of filtered triples: {len(filtered_english_triples):,}')

Size of filtered triples: 1,109,340


In [20]:
filtered_english_triples_sorted = filtered_english_triples.sort_values(by='edge_weight', ascending=False)

In [21]:
# Initialize and run
print("🚀 ULTRA FAST MODE - Processing all 1.6M triples!")
ultra_trainer = AdaptiveTrainingLoop(mvp_agent, filtered_english_triples_sorted, ultra_fast_config)
# training_history = ultra_trainer.train_until_convergence()

🚀 ULTRA FAST MODE - Processing all 1.6M triples!
🎯 Adaptive Training Loop Initialized!
   Dataset size: 1,109,340 triples
   Starting batch size: 50,000
   Quality threshold: 0.700


## Numpy Version
***
Training loop optimized as numpy version

In [34]:
import numpy as np
from numba import jit, prange
import time
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class HyperNumpyTrainingLoop:
    def __init__(self, data: np.ndarray, hyperparams: Optional[Dict] = None):
        """Initialize with dense matrices for Numba compatibility"""
        self.data = data.astype(np.float32)
        self.n_triples = len(data)
        
        self.hp = self._init_hyperparams(hyperparams)
        
        # Use dense matrix for Numba compatibility
        print(f"🔧 Using dense matrix for {self.hp['max_nodes']:,} nodes")
        self.graph_adjacency = np.zeros(
            (self.hp['max_nodes'], self.hp['max_nodes']), 
            dtype=np.float32
        )
        
        # Keep these as regular arrays (much smaller)
        self.node_degrees = np.zeros(self.hp['max_nodes'], dtype=np.int32)
        self.edge_counts = np.zeros(self.hp['max_relations'], dtype=np.int32)
        
        # Training state arrays
        self.validation_scores = np.zeros(self.n_triples, dtype=np.float32)
        self.processing_order = np.arange(self.n_triples)
        self.processed_mask = np.zeros(self.n_triples, dtype=bool)
        
        # Performance tracking
        self.metrics = {
            'edges_added': 0,
            'edges_rejected': 0,
            'processing_time': 0,
            'validation_time': 0,
            'update_time': 0
        }
        
        print(f"🚀 HyperNumpy Training Loop Initialized")
        print(f"   Data shape: {data.shape}")
        print(f"   Max nodes: {self.hp['max_nodes']:,}")
        print(f"   Batch size: {self.hp['batch_size']:,}")
        print(f"   Using {'GPU' if self.hp['use_gpu'] else 'CPU'} acceleration")
        
    def _init_hyperparams(self, custom_hp: Optional[Dict]) -> Dict:
        """Initialize hyperparameters with defaults"""
        defaults = {
            # Core training parameters
            'batch_size': 10000,
            'max_epochs': 10,
            'learning_rate': 1.0,
            'learning_rate_decay': 0.95,
            
            # Quality thresholds
            'quality_threshold_start': 0.3,
            'quality_threshold_end': 0.7,
            'quality_threshold_schedule': 'linear',  # linear, exponential, cosine, step
            
            # Validation parameters
            'validation_weight_factor': 0.4,
            'validation_connectivity_factor': 0.3,
            'validation_novelty_factor': 0.2,
            'validation_frequency_factor': 0.1,
            
            # Batch selection strategy
            'batch_selection': 'priority',  # random, priority, adaptive, curriculum
            'priority_decay': 0.99,
            'curriculum_start_easy': True,
            
            # Graph constraints
            'max_nodes': 100000,
            'max_relations': 100,
            'max_degree': 1000,
            'allow_self_loops': False,
            'symmetric_relations': False,
            
            # Performance optimizations
            'use_numba': True,
            'use_gpu': False,  # Requires CuPy
            'parallel_validation': True,
            'chunk_size': 1000,
            'prefetch_batches': 2,
            
            # Convergence criteria
            'convergence_window': 5,
            'convergence_threshold': 0.001,
            'early_stopping_patience': 3,
            'min_improvement': 0.0001,
            
            # Memory management
            'sparse_threshold': 0.1,  # Switch to sparse at this density
            'compression_level': 0,  # 0=none, 1=light, 2=aggressive
            'gc_frequency': 10000,  # Garbage collect every N operations
            
            # Advanced features
            'adaptive_batch_sizing': True,
            'dynamic_threshold_adjustment': True,
            'importance_sampling': True,
            'negative_sampling_ratio': 0.0,
            'dropout_rate': 0.0,
            
            # Monitoring
            'verbose': 1,
            'log_frequency': 1000,
            'checkpoint_frequency': 10000,
            'profile_performance': False
        }
        
        if custom_hp:
            defaults.update(custom_hp)
        
        return defaults
    
    @staticmethod
    @jit(nopython=True, parallel=True, cache=True)
    def _fast_validate_batch(data_batch, graph_adj, node_degrees, edge_counts,
                           weight_factor, conn_factor, novel_factor, freq_factor):
        """
        Numba-accelerated batch validation
        """
        n_batch = len(data_batch)
        scores = np.zeros(n_batch, dtype=np.float32)
        
        for i in prange(n_batch):
            start_idx = int(data_batch[i, 0])
            rel_idx = int(data_batch[i, 1])
            end_idx = int(data_batch[i, 2])
            weight = data_batch[i, 3]
            
            # Weight component
            score = weight * weight_factor
            
            # Connectivity component
            start_degree = node_degrees[start_idx]
            end_degree = node_degrees[end_idx]
            connectivity = (start_degree + end_degree) / (start_degree + end_degree + 10)
            score += connectivity * conn_factor
            
            # Novelty component (inverse of existing edge weight)
            existing_weight = graph_adj[start_idx, end_idx]
            novelty = 1.0 - existing_weight
            score += novelty * novel_factor
            
            # Frequency component (prefer rare relations)
            rel_freq = edge_counts[rel_idx]
            freq_score = 1.0 / (1.0 + np.log1p(rel_freq))
            score += freq_score * freq_factor
            
            scores[i] = score
        
        return scores
    
    def compute_validation_scores(self, batch_indices: np.ndarray) -> np.ndarray:
        """
        Compute validation scores for a batch of triples
        """
        batch_data = self.data[batch_indices]
        
        # Always use NumPy fallback for now (Numba has sparse matrix issues)
        if self.hp['use_numba'] and self.hp['parallel_validation'] and False:  # Disabled Numba for now
            scores = self._fast_validate_batch(
                batch_data, 
                self.graph_adjacency,
                self.node_degrees,
                self.edge_counts,
                self.hp['validation_weight_factor'],
                self.hp['validation_connectivity_factor'],
                self.hp['validation_novelty_factor'],
                self.hp['validation_frequency_factor']
            )
        else:
            # Use vectorized NumPy (still very fast)
            scores = self._numpy_validate_batch(batch_data)
        
        return scores
    
    def _numpy_validate_batch(self, batch_data: np.ndarray) -> np.ndarray:
        """
        Pure NumPy validation (fallback)
        """
        start_indices = batch_data[:, 0].astype(int)
        rel_indices = batch_data[:, 1].astype(int)
        end_indices = batch_data[:, 2].astype(int)
        weights = batch_data[:, 3]
        
        # Vectorized score computation
        scores = weights * self.hp['validation_weight_factor']
        
        # Connectivity scores
        start_degrees = self.node_degrees[start_indices]
        end_degrees = self.node_degrees[end_indices]
        connectivity = (start_degrees + end_degrees) / (start_degrees + end_degrees + 10)
        scores += connectivity * self.hp['validation_connectivity_factor']
        
        # Novelty scores
        existing_weights = self.graph_adjacency[start_indices, end_indices]
        novelty = 1.0 - existing_weights
        scores += novelty * self.hp['validation_novelty_factor']
        
        # Frequency scores
        rel_freqs = self.edge_counts[rel_indices]
        freq_scores = 1.0 / (1.0 + np.log1p(rel_freqs))
        scores += freq_scores * self.hp['validation_frequency_factor']
        
        return scores
    
    def select_batch(self, epoch: int, remaining_indices: np.ndarray) -> np.ndarray:
        """
        Select next batch based on strategy
        """
        n_remaining = len(remaining_indices)
        batch_size = int(min(self.hp['batch_size'], n_remaining))  # Ensure int
        
        if self.hp['batch_selection'] == 'random':
            return np.random.choice(remaining_indices, batch_size, replace=False)
        
        elif self.hp['batch_selection'] == 'priority':
            # Score all remaining triples
            scores = self.validation_scores[remaining_indices]
            
            # Add some randomness to avoid getting stuck
            noise = np.random.normal(0, 0.1, len(scores))
            scores = scores + noise
            
            # Select top scoring
            top_indices = np.argpartition(scores, -batch_size)[-batch_size:]
            return remaining_indices[top_indices]
        
        elif self.hp['batch_selection'] == 'curriculum':
            # Start with easy (high weight) triples, progress to harder
            progress = epoch / self.hp['max_epochs']
            
            if self.hp['curriculum_start_easy']:
                # Easy to hard
                threshold = 1.0 - progress * 0.7
            else:
                # Hard to easy
                threshold = 0.3 + progress * 0.7
            
            weights = self.data[remaining_indices, 3]
            if self.hp['curriculum_start_easy']:
                valid_mask = weights >= threshold
            else:
                valid_mask = weights <= threshold
            
            valid_indices = remaining_indices[valid_mask]
            if len(valid_indices) < batch_size:
                valid_indices = remaining_indices
            
            return np.random.choice(valid_indices, min(batch_size, len(valid_indices)), replace=False)
        
        elif self.hp['batch_selection'] == 'adaptive':
            # Dynamically adjust strategy based on acceptance rate
            if self.metrics['edges_added'] > 0:
                acceptance_rate = self.metrics['edges_added'] / (self.metrics['edges_added'] + self.metrics['edges_rejected'])
            else:
                acceptance_rate = 0.5
            
            if acceptance_rate < 0.3:
                # Low acceptance - try easier triples
                weights = self.data[remaining_indices, 3]
                top_weight_indices = np.argpartition(weights, -batch_size)[-batch_size:]
                return remaining_indices[top_weight_indices]
            else:
                # Good acceptance - explore more
                return np.random.choice(remaining_indices, batch_size, replace=False)
    
    def update_graph(self, batch_indices: np.ndarray, scores: np.ndarray, threshold: float) -> int:
        """
        Update graph with validated triples
        """
        accepted_mask = scores >= threshold
        accepted_indices = batch_indices[accepted_mask]
        
        n_accepted = 0
        for idx in accepted_indices:
            start_idx = int(self.data[idx, 0])
            rel_idx = int(self.data[idx, 1])
            end_idx = int(self.data[idx, 2])
            weight = self.data[idx, 3]
            
            # Check constraints
            if self.node_degrees[start_idx] >= self.hp['max_degree']:
                continue
            if self.node_degrees[end_idx] >= self.hp['max_degree']:
                continue
            if not self.hp['allow_self_loops'] and start_idx == end_idx:
                continue
            
            # Update adjacency matrix
            self.graph_adjacency[start_idx, end_idx] = max(
                self.graph_adjacency[start_idx, end_idx], 
                weight
            )
            
            # Update symmetric if needed
            if self.hp['symmetric_relations']:
                self.graph_adjacency[end_idx, start_idx] = self.graph_adjacency[start_idx, end_idx]
            
            # Update degrees and counts
            if self.graph_adjacency[start_idx, end_idx] == weight:  # New edge
                self.node_degrees[start_idx] += 1
                self.node_degrees[end_idx] += 1
                self.edge_counts[rel_idx] += 1
                n_accepted += 1
        
        # Mark as processed
        self.processed_mask[batch_indices] = True
        
        return n_accepted
    
    def get_quality_threshold(self, epoch: int) -> float:
        """
        Calculate quality threshold for current epoch
        """
        progress = epoch / max(1, self.hp['max_epochs'] - 1)
        start = self.hp['quality_threshold_start']
        end = self.hp['quality_threshold_end']
        
        if self.hp['quality_threshold_schedule'] == 'linear':
            return start + progress * (end - start)
        
        elif self.hp['quality_threshold_schedule'] == 'exponential':
            return start * (end / start) ** progress
        
        elif self.hp['quality_threshold_schedule'] == 'cosine':
            return start + (end - start) * 0.5 * (1 - np.cos(np.pi * progress))
        
        elif self.hp['quality_threshold_schedule'] == 'step':
            steps = 5
            step_size = (end - start) / steps
            return start + int(progress * steps) * step_size
        
        return start
    
    def train(self) -> Dict:
        """
        Main training loop - optimized for speed
        Stops if:
          - max_epochs reached
          - acceptance rate < 1%
          - all nodes included
          - new nodes added in epoch < min_new_nodes_per_epoch
        """
        print(f"\n{'='*60}")
        print(f"🏃 STARTING HYPERPARAMETER TRAINING")
        print(f"{'='*60}")

        min_new_nodes_per_epoch = self.hp.get('min_new_nodes_per_epoch', 1)

        # Pre-compute all validation scores
        print("📊 Pre-computing validation scores...")
        start_time = time.time()

        for i in range(0, self.n_triples, self.hp['chunk_size']):
            chunk_indices = np.arange(i, min(i + self.hp['chunk_size'], self.n_triples))
            self.validation_scores[chunk_indices] = self.compute_validation_scores(chunk_indices)
            if i % (self.hp['chunk_size'] * 10) == 0:
                print(f"   Processed {i:,}/{self.n_triples:,} triples...")

        precompute_time = time.time() - start_time
        print(f"   ✅ Pre-computation done in {precompute_time:.1f}s")

        unique_start_nodes = set(self.data[:, 0].astype(int))
        unique_end_nodes = set(self.data[:, 2].astype(int))
        all_unique_nodes = unique_start_nodes.union(unique_end_nodes)
        total_unique_nodes = len(all_unique_nodes)
        print(f"   📊 Dataset contains {total_unique_nodes:,} unique nodes")

        history = []
        convergence_scores = []

        prev_active_nodes_count = 0

        for epoch in range(self.hp['max_epochs']):
            epoch_start = time.time()
            epoch_stats = {
                'epoch': epoch,
                'accepted': 0,
                'rejected': 0,
                'threshold': self.get_quality_threshold(epoch)
            }

            print(f"\n📅 EPOCH {epoch + 1}/{self.hp['max_epochs']}")
            print(f"   Quality threshold: {epoch_stats['threshold']:.3f}")

            remaining_mask = ~self.processed_mask
            remaining_indices = np.where(remaining_mask)[0]

            if len(remaining_indices) == 0:
                print("   ✅ All triples processed!")
                break

            print(f"   Remaining triples: {len(remaining_indices):,}")

            n_batches = 0
            batch_times = []

            while len(remaining_indices) > 0:
                batch_start = time.time()
                batch_indices = self.select_batch(epoch, remaining_indices)
                scores = self.validation_scores[batch_indices]
                n_accepted = self.update_graph(batch_indices, scores, epoch_stats['threshold'])
                epoch_stats['accepted'] += n_accepted
                epoch_stats['rejected'] += len(batch_indices) - n_accepted
                self.metrics['edges_added'] += n_accepted
                self.metrics['edges_rejected'] += len(batch_indices) - n_accepted
                remaining_mask = ~self.processed_mask
                remaining_indices = np.where(remaining_mask)[0]
                batch_time = time.time() - batch_start
                batch_times.append(batch_time)
                n_batches += 1
                if n_batches % max(1, 100000 // self.hp['batch_size']) == 0:
                    acceptance_rate = n_accepted / max(1, len(batch_indices))
                    processing_rate = len(batch_indices) / max(batch_time, 1e-6)
                    print(f"      Batch {n_batches}: {n_accepted}/{len(batch_indices)} accepted "
                          f"({acceptance_rate:.3f}), {processing_rate:.0f} triples/sec")
                if self.hp['adaptive_batch_sizing'] and n_batches % 10 == 0:
                    if epoch_stats['accepted'] / max(1, epoch_stats['accepted'] + epoch_stats['rejected']) > 0.8:
                        self.hp['batch_size'] = int(min(self.hp['batch_size'] * 1.2, 50000))
                    elif epoch_stats['accepted'] / max(1, epoch_stats['accepted'] + epoch_stats['rejected']) < 0.2:
                        self.hp['batch_size'] = int(max(self.hp['batch_size'] * 0.8, 1000))

            epoch_time = time.time() - epoch_start
            total_processed = epoch_stats['accepted'] + epoch_stats['rejected']
            acceptance_rate = epoch_stats['accepted'] / max(1, total_processed)

            # Calculate active nodes
            active_nodes = set(np.where(self.node_degrees > 0)[0])
            active_nodes_count = len(active_nodes)
            active_nodes_pct = (active_nodes_count / total_unique_nodes) * 100
            new_nodes_added = active_nodes_count - prev_active_nodes_count
            prev_active_nodes_count = active_nodes_count

            epoch_stats.update({
                'duration': epoch_time,
                'acceptance_rate': acceptance_rate,
                'processing_rate': total_processed / epoch_time,
                'total_edges': np.sum(self.graph_adjacency > 0),
                'avg_degree': np.mean(self.node_degrees[self.node_degrees > 0]) if np.any(self.node_degrees > 0) else 0,
                'active_nodes': active_nodes_count,
                'new_nodes_added': new_nodes_added
            })

            history.append(epoch_stats)
            convergence_scores.append(acceptance_rate)

            print(f"\n   📊 EPOCH SUMMARY:")
            print(f"      Duration: {epoch_time:.1f}s")
            print(f"      Acceptance rate: {acceptance_rate:.3f}")
            print(f"      Processing rate: {epoch_stats['processing_rate']:.0f} triples/sec")
            print(f"      Total edges: {epoch_stats['total_edges']:,}")
            print(f"      Average degree: {epoch_stats['avg_degree']:.1f}")
            print(f"      Active nodes: {active_nodes_count:,}/{total_unique_nodes:,} ({active_nodes_pct:.1f}%)")
            print(f"      New nodes added this epoch: {new_nodes_added}")

            # Stop if acceptance rate is very low
            if acceptance_rate < 0.01:
                print(f"\n🛑 EARLY STOPPING: Acceptance rate below 1% ({acceptance_rate:.4f})")
                break

            # Stop if all nodes included
            if active_nodes_count >= total_unique_nodes:
                print(f"\n🎯 EARLY STOPPING: All nodes included in the graph!")
                break

            # Stop if not enough new nodes added
            if new_nodes_added < min_new_nodes_per_epoch:
                print(f"\n🛑 EARLY STOPPING: Less than {min_new_nodes_per_epoch} new nodes added in this epoch.")
                break

            # Convergence check (unchanged)
            if len(convergence_scores) >= self.hp['convergence_window']:
                recent_scores = convergence_scores[-self.hp['convergence_window']:]
                score_std = np.std(recent_scores)
                score_trend = np.polyfit(range(len(recent_scores)), recent_scores, 1)[0]
                if score_std < self.hp['convergence_threshold'] and abs(score_trend) < self.hp['min_improvement']:
                    print(f"\n🎯 CONVERGENCE DETECTED!")
                    print(f"   Score stability: {score_std:.4f}")
                    print(f"   Score trend: {score_trend:.6f}")
                    break

            if self.hp['learning_rate_decay'] < 1.0:
                self.hp['learning_rate'] *= self.hp['learning_rate_decay']

            if epoch < self.hp['max_epochs'] - 1:
                self.processed_mask.fill(False)

        total_time = sum(h['duration'] for h in history)
        print(f"\n{'='*60}")
        print(f"🎉 TRAINING COMPLETE!")
        print(f"{'='*60}")
        print(f"Total time: {total_time:.1f}s")
        print(f"Total edges: {np.sum(self.graph_adjacency > 0):,}")
        print(f"Average processing rate: {self.n_triples / total_time:.0f} triples/sec")

        return {
            'history': history,
            'graph': self.graph_adjacency,
            'node_degrees': self.node_degrees,
            'edge_counts': self.edge_counts,
            'metrics': self.metrics
        }
    
    def get_graph_stats(self) -> Dict:
        """Get current graph statistics"""
        mask = self.graph_adjacency > 0
        n_edges = np.sum(mask)
        
        if n_edges == 0:
            return {
                'n_edges': 0,
                'n_nodes': 0,
                'density': 0,
                'avg_weight': 0,
                'avg_degree': 0
            }
        
        active_nodes = np.unique(np.concatenate([
            np.where(self.node_degrees > 0)[0],
            np.where(np.sum(self.graph_adjacency, axis=0) > 0)[0]
        ]))
        
        return {
            'n_edges': n_edges,
            'n_nodes': len(active_nodes),
            'density': n_edges / (len(active_nodes) ** 2) if len(active_nodes) > 0 else 0,
            'avg_weight': np.mean(self.graph_adjacency[mask]),
            'avg_degree': np.mean(self.node_degrees[active_nodes]) if len(active_nodes) > 0 else 0,
            'max_degree': np.max(self.node_degrees),
            'sparsity': 1 - (n_edges / self.graph_adjacency.size)
        }


def prepare_conceptnet_for_numpy(df, concept_to_idx=None, relation_to_idx=None):
    """
    Convert ConceptNet DataFrame to NumPy format
    """
    print("🔄 Converting ConceptNet to NumPy format...")
    
    # Create mappings if not provided
    if concept_to_idx is None:
        all_concepts = pd.concat([df['start_concept'], df['end_concept']]).unique()
        concept_to_idx = {c: i for i, c in enumerate(all_concepts)}
    
    if relation_to_idx is None:
        relation_to_idx = {r: i for i, r in enumerate(df['relation_type'].unique())}
    
    # Convert to indices
    start_indices = df['start_concept'].map(concept_to_idx).values
    relation_indices = df['relation_type'].map(relation_to_idx).values
    end_indices = df['end_concept'].map(concept_to_idx).values
    weights = df['edge_weight'].values
    
    # Stack into array
    data = np.column_stack([start_indices, relation_indices, end_indices, weights])
    
    print(f"   ✅ Converted {len(data):,} triples")
    print(f"   Unique concepts: {len(concept_to_idx):,}")
    print(f"   Unique relations: {len(relation_to_idx):,}")
    
    return data, concept_to_idx, relation_to_idx


# Example usage:
if __name__ == "__main__":
    # Example with synthetic data
    print("Creating example dataset...")
    n_samples = 100000
    n_concepts = 10000
    n_relations = 50
    
    data = np.column_stack([
        np.random.randint(0, n_concepts, n_samples),  # start
        np.random.randint(0, n_relations, n_samples),  # relation
        np.random.randint(0, n_concepts, n_samples),  # end
        np.random.rand(n_samples)  # weights
    ])
    
    # Configure hyperparameters
    hyperparams = {
        'batch_size': 5000,
        'max_epochs': 5,
        'batch_selection': 'adaptive',
        'quality_threshold_schedule': 'cosine',
        'use_numba': True,
        'max_nodes': n_concepts,
        'max_relations': n_relations
    }
    
    # Train
    trainer = HyperNumpyTrainingLoop(data, hyperparams)
    results = trainer.train()
    
    # Get final stats
    final_stats = trainer.get_graph_stats()
    print(f"\nFinal graph statistics:")
    for key, value in final_stats.items():
        print(f"   {key}: {value:.3f}" if isinstance(value, float) else f"   {key}: {value:,}")

Creating example dataset...
🔧 Using dense matrix for 10,000 nodes
🚀 HyperNumpy Training Loop Initialized
   Data shape: (100000, 4)
   Max nodes: 10,000
   Batch size: 5,000
   Using CPU acceleration

🏃 STARTING HYPERPARAMETER TRAINING
📊 Pre-computing validation scores...
   Processed 0/100,000 triples...
   Processed 10,000/100,000 triples...
   Processed 20,000/100,000 triples...
   Processed 30,000/100,000 triples...
   Processed 40,000/100,000 triples...
   Processed 50,000/100,000 triples...
   Processed 60,000/100,000 triples...
   Processed 70,000/100,000 triples...
   Processed 80,000/100,000 triples...
   Processed 90,000/100,000 triples...
   ✅ Pre-computation done in 0.1s
   📊 Dataset contains 10,000 unique nodes

📅 EPOCH 1/5
   Quality threshold: 0.300
   Remaining triples: 100,000
      Batch 16: 5996/6000 accepted (0.999), 600015 triples/sec

   📊 EPOCH SUMMARY:
      Duration: 0.2s
      Acceptance rate: 1.000
      Processing rate: 586390 triples/sec
      Total edges: 

In [37]:
# # Step 1: Clean your ConceptNet data first (using the detector from earlier)
# print("🧹 Cleaning ConceptNet data...")
# junk_chars = ['n', 'v', 'r', 'o', 't', 's', 'y', 'c', 'e', 'u']
# # REmove start and end concepts that are only junk characters
# for junk in junk_chars:
#     filtered_english_triples = cleaned_english_triples[~cleaned_english_triples['start_concept'].str.contains(junk)]
#     filtered_english_triples = filtered_english_triples[~filtered_english_triples['end_concept'].str.contains(junk)]
    
# print(f'Size of filtered triples: {len(filtered_english_triples):,}')
# cleaned_df = filtered_english_triples.copy()

cleaned_df = cleaned_english_triples.copy()
print(f"✅ Cleaned data: {len(cleaned_df):,} triples")

# Step 2: Convert to NumPy format
print("\n🔄 Converting to NumPy format...")
numpy_data, concept_map, relation_map = prepare_conceptnet_for_numpy(cleaned_df)

# Step 3: Configure hyperparameters for your real data
# Modify your hyperparameters to use fewer nodes
hyperparams = {
    'batch_size': 10000,
    'max_epochs': 25,
    'quality_threshold_start': 0.2,
    'quality_threshold_end': 0.5,
    'quality_threshold_schedule': 'linear',
    'min_new_nodes_per_epoch': 1,
    # Disable Numba for compatibility
    'use_numba': False,
    'parallel_validation': False,

    'max_nodes': 10000,  # <-- Reduce from 50000 to 10000
    'max_relations': len(relation_map),
    'max_degree': 5000,
    'allow_self_loops': False,
    'validation_weight_factor': 0.5,
    'validation_connectivity_factor': 0.3,
    'validation_novelty_factor': 0.1,
    'validation_frequency_factor': 0.1,
    'chunk_size': 10000,
    'adaptive_batch_sizing': True,
    'convergence_window': 5,
    'convergence_threshold': 0.01,
    'early_stopping_patience': 3,
    'verbose': 1,
    'log_frequency': 1000,
}

# Filter your data to only use the top 10,000 most common concepts
print("🔧 Filtering to most common concepts...")
concept_counts = pd.concat([
    cleaned_df['start_concept'].value_counts(),
    cleaned_df['end_concept'].value_counts()
]).groupby(level=0).sum().sort_values(ascending=False)

top_concepts = set(concept_counts.head(10000).index)  # <-- Reduce from 50000 to 10000
filtered_df = cleaned_df[
    cleaned_df['start_concept'].isin(top_concepts) & 
    cleaned_df['end_concept'].isin(top_concepts)
]

print(f"✅ Filtered from {len(cleaned_df):,} to {len(filtered_df):,} triples")
print(f"✅ Using top {len(top_concepts):,} most common concepts")

# Convert the filtered data
numpy_data, concept_map, relation_map = prepare_conceptnet_for_numpy(filtered_df)

# Step 4: Train!
print("\n🚀 Starting training...")
trainer = HyperNumpyTrainingLoop(numpy_data, hyperparams)
results = trainer.train()

# Step 5: Extract results
print("\n📊 Extracting results...")
adjacency_matrix = results['graph']
history = results['history']

✅ Cleaned data: 1,655,522 triples

🔄 Converting to NumPy format...
🔄 Converting ConceptNet to NumPy format...
   ✅ Converted 1,655,522 triples
   Unique concepts: 754,380
   Unique relations: 47
🔧 Filtering to most common concepts...
✅ Filtered from 1,655,522 to 214,542 triples
✅ Using top 10,000 most common concepts
🔄 Converting ConceptNet to NumPy format...
   ✅ Converted 214,542 triples
   Unique concepts: 9,993
   Unique relations: 47

🚀 Starting training...
🔧 Using dense matrix for 10,000 nodes
🚀 HyperNumpy Training Loop Initialized
   Data shape: (214542, 4)
   Max nodes: 10,000
   Batch size: 10,000
   Using CPU acceleration

🏃 STARTING HYPERPARAMETER TRAINING
📊 Pre-computing validation scores...
   Processed 0/214,542 triples...
   Processed 100,000/214,542 triples...
   Processed 200,000/214,542 triples...
   ✅ Pre-computation done in 0.0s
   📊 Dataset contains 9,993 unique nodes

📅 EPOCH 1/25
   Quality threshold: 0.200
   Remaining triples: 214,542
      Batch 10: 5172/10000

# Phase 7
***
Testing suite for current trained knowledge graph agent

In [24]:
class NumpyKnowledgeGraphInterface:
    """
    Interface to work with your numpy-trained knowledge graph
    """
    
    def __init__(self, adjacency_matrix, concept_map, relation_map, results):
        self.adjacency_matrix = adjacency_matrix
        self.concept_map = concept_map  # concept_name -> index
        self.relation_map = relation_map  # relation_name -> index
        self.results = results
        
        # Create reverse mappings
        self.idx_to_concept = {idx: concept for concept, idx in concept_map.items()}
        self.idx_to_relation = {idx: relation for relation, idx in relation_map.items()}
        
        # Get active nodes (nodes that actually have connections)
        self.active_mask = (adjacency_matrix > 0).any(axis=0) | (adjacency_matrix > 0).any(axis=1)
        self.active_indices = np.where(self.active_mask)[0]
        
        print(f"🧠 Numpy Knowledge Graph Interface Initialized!")
        print(f"   Total concepts: {len(concept_map):,}")
        print(f"   Active concepts: {len(self.active_indices):,}")
        print(f"   Total relations: {len(relation_map):,}")
        print(f"   Graph edges: {np.sum(adjacency_matrix > 0):,}")
        
    def get_concept_neighbors(self, concept_name, max_results=10):
        """Get neighbors of a concept"""
        if concept_name not in self.concept_map:
            similar = self.find_similar_concepts(concept_name)
            print(f"❓ '{concept_name}' not found. Similar concepts: {similar[:5]}")
            return []
        
        concept_idx = self.concept_map[concept_name]
        neighbors = []
        
        # Outgoing connections (concept -> others)
        outgoing_indices = np.where(self.adjacency_matrix[concept_idx, :] > 0)[0]
        for target_idx in outgoing_indices[:max_results//2]:
            weight = self.adjacency_matrix[concept_idx, target_idx]
            target_concept = self.idx_to_concept.get(target_idx, f"concept_{target_idx}")
            neighbors.append({
                'from': concept_name,
                'to': target_concept,
                'weight': float(weight),
                'direction': 'outgoing'
            })
        
        # Incoming connections (others -> concept)
        incoming_indices = np.where(self.adjacency_matrix[:, concept_idx] > 0)[0]
        for source_idx in incoming_indices[:max_results//2]:
            weight = self.adjacency_matrix[source_idx, concept_idx]
            source_concept = self.idx_to_concept.get(source_idx, f"concept_{source_idx}")
            neighbors.append({
                'from': source_concept,
                'to': concept_name,
                'weight': float(weight),
                'direction': 'incoming'
            })
        
        return neighbors
    
    def explore_concept(self, concept_name, max_results=10):
        """Explore a concept and its relationships"""
        print(f"🔍 EXPLORING: '{concept_name}'")
        print("="*50)
        
        neighbors = self.get_concept_neighbors(concept_name, max_results)
        
        if not neighbors:
            print(f"❌ No connections found for '{concept_name}'")
            return None
        
        print(f"📍 CONNECTIONS for '{concept_name}':")
        for neighbor in neighbors:
            direction_symbol = "-->" if neighbor['direction'] == 'outgoing' else "<--"
            other_concept = neighbor['to'] if neighbor['direction'] == 'outgoing' else neighbor['from']
            print(f"   {concept_name} {direction_symbol} {other_concept} (weight: {neighbor['weight']:.3f})")
        
        return neighbors
    
    def find_connection(self, concept1, concept2, max_path_length=3):
        """Find if two concepts are connected"""
        if concept1 not in self.concept_map or concept2 not in self.concept_map:
            missing = [c for c in [concept1, concept2] if c not in self.concept_map]
            print(f"❌ Concepts not found: {missing}")
            return None
        
        idx1 = self.concept_map[concept1]
        idx2 = self.concept_map[concept2]
        
        print(f"🧠 ANALYZING CONNECTION: '{concept1}' ↔ '{concept2}'")
        print("="*60)
        
        # Check direct connection
        direct_weight = self.adjacency_matrix[idx1, idx2]
        reverse_weight = self.adjacency_matrix[idx2, idx1]
        
        if direct_weight > 0:
            print(f"✅ DIRECT CONNECTION: {concept1} --> {concept2} (weight: {direct_weight:.3f})")
            return {'type': 'direct', 'weight': float(direct_weight), 'direction': 'forward'}
        
        if reverse_weight > 0:
            print(f"✅ REVERSE CONNECTION: {concept2} --> {concept1} (weight: {reverse_weight:.3f})")
            return {'type': 'direct', 'weight': float(reverse_weight), 'direction': 'reverse'}
        
        # Check for shared connections
        neighbors1 = set(np.concatenate([
            np.where(self.adjacency_matrix[idx1, :] > 0)[0],
            np.where(self.adjacency_matrix[:, idx1] > 0)[0]
        ]))
        
        neighbors2 = set(np.concatenate([
            np.where(self.adjacency_matrix[idx2, :] > 0)[0],
            np.where(self.adjacency_matrix[:, idx2] > 0)[0]
        ]))
        
        shared = neighbors1 & neighbors2
        if shared:
            shared_concepts = [self.idx_to_concept.get(idx, f"concept_{idx}") for idx in list(shared)[:5]]
            print(f"🤝 SHARED CONNECTIONS: Both connected to {shared_concepts}")
            return {'type': 'shared', 'shared_concepts': shared_concepts, 'count': len(shared)}
        
        print(f"❌ NO CONNECTION FOUND between '{concept1}' and '{concept2}'")
        return None
    
    def find_similar_concepts(self, concept_name, max_results=5):
        """Find concepts with similar names"""
        concept_lower = concept_name.lower()
        similar = []
        
        for concept in self.concept_map.keys():
            concept_lower_check = concept.lower()
            if (concept_lower in concept_lower_check or 
                concept_lower_check in concept_lower or
                abs(len(concept_lower) - len(concept_lower_check)) <= 2):
                similar.append(concept)
                if len(similar) >= max_results:
                    break
        
        return similar
    
    def get_top_concepts(self, n=20):
        """Get most connected concepts"""
        # Calculate degree for each active concept
        degrees = {}
        for idx in self.active_indices:
            degree = np.sum(self.adjacency_matrix[idx, :] > 0) + np.sum(self.adjacency_matrix[:, idx] > 0)
            concept_name = self.idx_to_concept.get(idx, f"concept_{idx}")
            degrees[concept_name] = degree
        
        # Sort by degree
        top_concepts = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:n]
        
        print(f"🔗 TOP {n} MOST CONNECTED CONCEPTS:")
        for concept, degree in top_concepts:
            print(f"   {concept}: {degree} connections")
        
        return top_concepts
    
    def analyze_graph_stats(self):
        """Analyze overall graph statistics"""
        print(f"\n📊 GRAPH ANALYSIS:")
        print("="*50)
        
        # Basic stats
        total_edges = np.sum(self.adjacency_matrix > 0)
        active_nodes = len(self.active_indices)
        
        print(f"📈 Basic Statistics:")
        print(f"   Total concepts: {len(self.concept_map):,}")
        print(f"   Active concepts: {active_nodes:,}")
        print(f"   Total edges: {total_edges:,}")
        print(f"   Graph density: {total_edges / (active_nodes * active_nodes) * 100:.4f}%")
        
        # Weight distribution
        weights = self.adjacency_matrix[self.adjacency_matrix > 0]
        print(f"\n⚖️  Weight Distribution:")
        print(f"   Mean weight: {np.mean(weights):.3f}")
        print(f"   Weight range: {np.min(weights):.3f} - {np.max(weights):.3f}")
        print(f"   High quality edges (>0.8): {np.sum(weights > 0.8):,}")
        
        # Degree distribution
        degrees = []
        for idx in self.active_indices:
            degree = np.sum(self.adjacency_matrix[idx, :] > 0) + np.sum(self.adjacency_matrix[:, idx] > 0)
            degrees.append(degree)
        
        print(f"\n🕸️  Connectivity:")
        print(f"   Average degree: {np.mean(degrees):.2f}")
        print(f"   Max degree: {np.max(degrees)}")
        print(f"   Isolated nodes: {np.sum(np.array(degrees) == 0):,}")
        
        return {
            'total_edges': int(total_edges),
            'active_nodes': active_nodes,
            'density': float(total_edges / (active_nodes * active_nodes)),
            'avg_weight': float(np.mean(weights)),
            'avg_degree': float(np.mean(degrees))
        }
    
    def random_exploration(self, n_concepts=5):
        """Explore random concepts to see what the graph learned"""
        print(f"\n🎲 RANDOM EXPLORATION:")
        print("="*50)
        
        # Get random active concepts
        random_indices = np.random.choice(self.active_indices, min(n_concepts, len(self.active_indices)), replace=False)
        
        for idx in random_indices:
            concept_name = self.idx_to_concept.get(idx, f"concept_{idx}")
            print(f"\n--- Random concept: '{concept_name}' ---")
            neighbors = self.get_concept_neighbors(concept_name, max_results=5)
            
            if neighbors:
                for neighbor in neighbors[:3]:  # Show top 3
                    direction_symbol = "-->" if neighbor['direction'] == 'outgoing' else "<--"
                    other_concept = neighbor['to'] if neighbor['direction'] == 'outgoing' else neighbor['from']
                    print(f"   {concept_name} {direction_symbol} {other_concept} (w: {neighbor['weight']:.3f})")
            else:
                print(f"   No connections found")

# Initialize the interface with your trained components
print("🚀 Creating Numpy Knowledge Graph Interface...")
kg_interface = NumpyKnowledgeGraphInterface(
    adjacency_matrix=results['graph'],
    concept_map=concept_map,
    relation_map=relation_map,
    results=results
)

# Quick demo of what you can do
print("\n" + "="*60)
print("🎯 TESTING YOUR TRAINED KNOWLEDGE GRAPH")
print("="*60)

# 1. Analyze overall statistics
stats = kg_interface.analyze_graph_stats()

# 2. Show top concepts
top_concepts = kg_interface.get_top_concepts(10)

# 3. Random exploration
kg_interface.random_exploration(3)

🚀 Creating Numpy Knowledge Graph Interface...
🧠 Numpy Knowledge Graph Interface Initialized!
   Total concepts: 49,969
   Active concepts: 46,852
   Total relations: 47
   Graph edges: 260,030

🎯 TESTING YOUR TRAINED KNOWLEDGE GRAPH

📊 GRAPH ANALYSIS:
📈 Basic Statistics:
   Total concepts: 49,969
   Active concepts: 46,852
   Total edges: 260,030
   Graph density: 0.0118%

⚖️  Weight Distribution:
   Mean weight: 0.942
   Weight range: 0.100 - 22.891
   High quality edges (>0.8): 161,956

🕸️  Connectivity:
   Average degree: 11.10
   Max degree: 4856
   Isolated nodes: 0
🔗 TOP 10 MOST CONNECTED CONCEPTS:
   r: 4856 connections
   en_1: 4849 connections
   a: 4822 connections
   en_2: 4781 connections
   v: 4725 connections
   n: 4661 connections
   en_3: 3487 connections
   person: 1636 connections
   en_4: 1182 connections
   people: 710 connections

🎲 RANDOM EXPLORATION:

--- Random concept: 'illegal' ---
   illegal --> illegal_alien (w: 0.250)
   illegal --> illegal_immigrant (w: 0.

In [25]:
# Test specific concepts
kg_interface.explore_concept('dog')
kg_interface.explore_concept('computer')
kg_interface.explore_concept('love')

# Test connections between concepts
kg_interface.find_connection('dog', 'animal')
kg_interface.find_connection('computer', 'technology')

# Explore what concepts are available
print("Available concepts sample:", list(concept_map.keys())[:20])

🔍 EXPLORING: 'dog'
📍 CONNECTIONS for 'dog':
   dog --> again (weight: 0.183)
   dog --> animal (weight: 9.410)
   dog --> bath (weight: 1.000)
   dog --> bed (weight: 2.000)
   dog --> big (weight: 0.782)
   dog <-- n (weight: 2.000)
   dog <-- en_1 (weight: 1.000)
   dog <-- v (weight: 1.000)
   dog <-- en_2 (weight: 1.000)
   dog <-- animal (weight: 3.129)
🔍 EXPLORING: 'computer'
📍 CONNECTIONS for 'computer':
   computer --> act (weight: 2.000)
   computer --> apartment (weight: 1.000)
   computer --> box (weight: 1.000)
   computer --> car (weight: 1.000)
   computer --> count (weight: 1.000)
   computer <-- r (weight: 1.000)
   computer <-- a (weight: 1.000)
   computer <-- v (weight: 1.000)
   computer <-- en_2 (weight: 1.000)
   computer <-- apple (weight: 4.164)
🔍 EXPLORING: 'love'
📍 CONNECTIONS for 'love':
   love --> around (weight: 0.157)
   love --> big (weight: 0.159)
   love --> blind (weight: 2.828)
   love --> care (weight: 0.156)
   love --> charity (weight: 1.000)
   l

In [28]:

# Interactive exploration
def interactive_test():
    """Simple interactive testing"""
    print("\n🎮 Interactive Testing (type 'quit' to exit)")
    
    while True:
        try:
            user_input = input("\n🤖 Enter concept to explore: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
                
            if not user_input:
                continue
                
            kg_interface.explore_concept(user_input)
            
        except KeyboardInterrupt:
            print("\n👋 Testing interrupted!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

print(f"\n🎯 Ready for testing! Try:")
print(f"   kg_interface.explore_concept('your_concept_here')")
print(f"   kg_interface.find_connection('concept1', 'concept2')")
print(f"   interactive_test()  # For interactive exploration")
# interactive_test()  # For interactive exploration


🎯 Ready for testing! Try:
   kg_interface.explore_concept('your_concept_here')
   kg_interface.find_connection('concept1', 'concept2')
   interactive_test()  # For interactive exploration


# Phase 8 
***
Saving the Agent and graph

In [29]:
class AgentSaveLoadManager:
    """
    Simple system to save and load your trained knowledge graph agent
    """
    
    def __init__(self, save_directory="../Data/Output/"):
        self.save_directory = save_directory
        os.makedirs(save_directory, exist_ok=True)
        print(f"💾 Save/Load Manager initialized")
        print(f"   Save directory: {save_directory}")
    
    def save_agent(self, kg_interface, filename_prefix="trained_agent"):
        """
        Save the complete trained agent state
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Package everything together
        agent_data = {
            # Core components
            'adjacency_matrix': kg_interface.adjacency_matrix,
            'concept_map': kg_interface.concept_map,
            'relation_map': kg_interface.relation_map,
            'results': kg_interface.results,
            
            # Derived components
            'idx_to_concept': kg_interface.idx_to_concept,
            'idx_to_relation': kg_interface.idx_to_relation,
            'active_mask': kg_interface.active_mask,
            'active_indices': kg_interface.active_indices,
            
            # Metadata
            'save_timestamp': timestamp,
            'stats': kg_interface.analyze_graph_stats(),
            'version': 'v1.0'
        }
        
        # Create filename
        filename = f"{filename_prefix}_{timestamp}.pkl"
        filepath = os.path.join(self.save_directory, filename)
        
        # Save
        print(f"💾 Saving trained agent...")
        with open(filepath, 'wb') as f:
            pickle.dump(agent_data, f)
        
        # Save metadata as JSON for easy inspection
        metadata = {
            'save_timestamp': timestamp,
            'total_concepts': len(agent_data['concept_map']),
            'active_concepts': len(agent_data['active_indices']),
            'total_edges': int(agent_data['stats']['total_edges']),
            'avg_degree': float(agent_data['stats']['avg_degree']),
            'file_size_mb': os.path.getsize(filepath) / (1024*1024),
            'version': agent_data['version']
        }
        
        metadata_file = filepath.replace('.pkl', '_metadata.json')
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ AGENT SAVED SUCCESSFULLY!")
        print(f"   File: {filepath}")
        print(f"   Size: {metadata['file_size_mb']:.1f} MB")
        print(f"   Concepts: {metadata['total_concepts']:,}")
        print(f"   Active concepts: {metadata['active_concepts']:,}")
        print(f"   Edges: {metadata['total_edges']:,}")
        print(f"   Metadata: {metadata_file}")
        
        return filepath
    
    def load_agent(self, filepath):
        """
        Load a previously saved agent
        """
        print(f"📂 Loading trained agent from: {filepath}")
        
        if not os.path.exists(filepath):
            print(f"❌ File not found: {filepath}")
            return None
        
        # Load the data
        with open(filepath, 'rb') as f:
            agent_data = pickle.load(f)
        
        # Recreate the interface
        kg_interface = NumpyKnowledgeGraphInterface(
            adjacency_matrix=agent_data['adjacency_matrix'],
            concept_map=agent_data['concept_map'],
            relation_map=agent_data['relation_map'],
            results=agent_data['results']
        )
        
        print(f"✅ AGENT LOADED SUCCESSFULLY!")
        print(f"   Saved: {agent_data['save_timestamp']}")
        print(f"   Version: {agent_data['version']}")
        print(f"   Concepts: {len(agent_data['concept_map']):,}")
        print(f"   Active concepts: {len(agent_data['active_indices']):,}")
        print(f"   Edges: {agent_data['stats']['total_edges']:,}")
        
        return kg_interface, agent_data
    
    def list_saved_agents(self):
        """
        List all saved agents in the directory
        """
        print(f"📋 SAVED AGENTS in {self.save_directory}:")
        print("="*60)
        
        agent_files = [f for f in os.listdir(self.save_directory) if f.endswith('.pkl')]
        
        if not agent_files:
            print("❌ No saved agents found")
            return []
        
        agents = []
        for filename in sorted(agent_files):
            filepath = os.path.join(self.save_directory, filename)
            metadata_file = filepath.replace('.pkl', '_metadata.json')
            
            # Try to load metadata
            if os.path.exists(metadata_file):
                with open(metadata_file, 'r') as f:
                    metadata = json.load(f)
                
                agents.append({
                    'filename': filename,
                    'filepath': filepath,
                    'metadata': metadata
                })
                
                print(f"\n📄 {filename}")
                print(f"   Saved: {metadata['save_timestamp']}")
                print(f"   Size: {metadata['file_size_mb']:.1f} MB")
                print(f"   Concepts: {metadata['total_concepts']:,}")
                print(f"   Edges: {metadata['total_edges']:,}")
            else:
                # Fallback - just show file info
                size_mb = os.path.getsize(filepath) / (1024*1024)
                agents.append({
                    'filename': filename,
                    'filepath': filepath,
                    'metadata': {'file_size_mb': size_mb}
                })
                
                print(f"\n📄 {filename} (no metadata)")
                print(f"   Size: {size_mb:.1f} MB")
        
        return agents
    
    def quick_save(self, kg_interface):
        """
        Quick save with automatic naming
        """
        return self.save_agent(kg_interface, "quick_save")
    
    def backup_save(self, kg_interface):
        """
        Create a backup with timestamp
        """
        return self.save_agent(kg_interface, "backup")

# Initialize the save/load manager
save_manager = AgentSaveLoadManager()

# Save your current trained agent
print("\n" + "="*60)
print("💾 SAVING YOUR TRAINED AGENT")
print("="*60)

saved_filepath = save_manager.save_agent(kg_interface, "conceptnet_trained")

# Test loading it back
print("\n" + "="*60)
print("📂 TESTING LOAD FUNCTIONALITY")
print("="*60)

loaded_kg_interface, loaded_data = save_manager.load_agent(saved_filepath)

# Verify it works
print("\n🔍 Testing loaded agent...")
loaded_kg_interface.explore_concept('dog')

print("\n📋 All saved agents:")
save_manager.list_saved_agents()

print(f"\n🎉 Save/Load system working perfectly!")
print(f"✅ Your agent is safely saved and can be reloaded anytime")
print(f"📁 Saved to: {saved_filepath}")

💾 Save/Load Manager initialized
   Save directory: ../Data/Output/

💾 SAVING YOUR TRAINED AGENT

📊 GRAPH ANALYSIS:
📈 Basic Statistics:
   Total concepts: 49,969
   Active concepts: 46,852
   Total edges: 260,030
   Graph density: 0.0118%

⚖️  Weight Distribution:
   Mean weight: 0.942
   Weight range: 0.100 - 22.891
   High quality edges (>0.8): 161,956

🕸️  Connectivity:
   Average degree: 11.10
   Max degree: 4856
   Isolated nodes: 0
💾 Saving trained agent...
✅ AGENT SAVED SUCCESSFULLY!
   File: ../Data/Output/conceptnet_trained_20250522_170059.pkl
   Size: 9538.4 MB
   Concepts: 49,969
   Active concepts: 46,852
   Edges: 260,030
   Metadata: ../Data/Output/conceptnet_trained_20250522_170059_metadata.json

📂 TESTING LOAD FUNCTIONALITY
📂 Loading trained agent from: ../Data/Output/conceptnet_trained_20250522_170059.pkl
🧠 Numpy Knowledge Graph Interface Initialized!
   Total concepts: 49,969
   Active concepts: 46,852
   Total relations: 47
   Graph edges: 260,030
✅ AGENT LOADED SUCCE